In [185]:
#Load environment variables from .env file
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import os
#for i in os.environ: print(i) ##test environment vars


from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re
from getpass import getpass

from pprint import pprint as pp
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 100)
from pprint import pprint
import ast
from scipy.stats import mode
import math
import traceback

def displays(df, n = 5):
    display(df.shape, df.head(n))

# Testing

## Azure

In [23]:
# set API keys and metadata
OPENAI_API_TYPE = os.environ["OPENAI_API_TYPE"] # azure
OPENAI_API_VERSION = os.environ["OPENAI_API_VERSION"] # 2023-03-15-preview
OPENAI_API_BASE = os.environ["OPENAI_API_BASE"] # https://w210-openai.openai.azure.com/
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] # key
OPENAI_AZURE_DEPLOYMENT_NAME = os.environ["OPENAI_AZURE_DEPLOYMENT_NAME"] # gpt-35-turbo
MODEL_NAME = "gpt-35-turbo"
SERPAPI_API_KEY = os.environ["SERPAPI_API_KEY"]
tool_names = ["serpapi"]

In [191]:
original_prediction

['A. blue',
 'A. blue',
 'A. blue',
 'A. blue.',
 'A. Blue',
 'A. blue',
 'A. blue',
 'A. blue.',
 'A. blue.',
 'A. Blue.']

In [147]:
# Create an instance of Azure OpenAI

llm = AzureChatOpenAI(
    deployment_name=OPENAI_AZURE_DEPLOYMENT_NAME,
    model_name=MODEL_NAME,
    temperature=0.3,
    max_tokens=25,
    request_timeout=20
)

In [150]:
# run a query
output = llm([
                HumanMessage(content="Return the letter B and nothing else.")
])

In [157]:
# show output
print(output)
print(output.content)

content='B' additional_kwargs={} example=False
B


## SerpApi

In [ ]:
# serpapi = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)

In [4]:
# create tool
tool_names = ["serpapi"]
tools = load_tools(tool_names)

In [9]:
# create agent
agent = initialize_agent(tools, llm, agent = "zero-shot-react-description", verbose = True)

In [11]:
# run agent
agent.run("Who will win the 2024 US Presidential election?")



> Entering new  chain...
 I need to find out who is running for the election
Action: Search
Action Input: "2024 US Presidential election candidates"
Observation: Four years after a historically large number of candidates ran for president, the field for the 2024 campaign is getting crowded once more, ...
Thought: I need to research the candidates and their chances of winning
Action: Search
Action Input: "2024 US Presidential election predictions"
Observation: The latest coverage of the 2024 presidential, House and Senate elections.
Thought: I now know the final answer
Final Answer: It is too early to predict who will win the 2024 US Presidential election.

> Finished chain.


'It is too early to predict who will win the 2024 US Presidential election.'

## Wikipedia

In [211]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

In [212]:
# create wikipedia instance
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [217]:
# run output
wikipedia.run("Inflation")

'Page: Inflation\nSummary: In economics, inflation is an increase in the general price level of goods and services in an economy. When the general price level rises, each unit of currency buys fewer goods and services; consequently, inflation corresponds to a reduction in the purchasing power of money. The opposite of inflation is deflation, a decrease in the general price level of goods and services. The common measure of inflation is the inflation rate, the annualized percentage change in a general price index. As prices faced by households do not all increase at the same rate, the consumer price index (CPI) is often used for this purpose. The employment cost index is also used for wages in the United States.\nThere is disagreement among economists as to the causes of inflation. Low or moderate inflation is widely attributed to fluctuations in real demand for goods and services or changes in available supplies such as during scarcities. Moderate inflation affects economies in both po

# Main

## Structure Data

In [24]:
class AutocastBuilder:
    """
    Build autocast dataframe to feed into ContextBuilder.
    """
    def __init__(self, data_path):
        self.data_path = data_path
        self.data = self.load_data()
        
    @staticmethod
    def clean_choices(choices):
        """lambda function for adding letter choices to 'choices' column in autocast"""
        if pd.notna(choices):
            choices = ast.literal_eval(choices)
            letters = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]
            choices_dict = zip(letters[:len(choices)], choices)
            return dict(choices_dict)
        else:
            return choices
        
    @staticmethod
    def get_crowd_avg(item):
        """Lambda function to Calculate average crowd forecast per question"""
        if pd.notna(item):
            items = ast.literal_eval(item)
            crowd_avg = np.mean([np.mean(item['forecast']) for item in items])
            return crowd_avg
        else:
            return np.nan
        
    @staticmethod
    def get_answer_choices(answer_positions, n_choices, t_f):
        """return list of choices given forecast values
        return [np.nan] if forecast is undecided
        handles instances of forecast as float instead of list of probabilities"""
        if isinstance(answer_positions, (int, float)):
            answer_positions = [answer_positions]

        if isinstance(answer_positions, list):
            answer_positions = np.array(answer_positions)
            
        #print(f"answer_positions: {answer_positions}")

        if answer_positions.size == 1:
            return [AutocastBuilder.get_single_answer(float(answer_positions[0]), n_choices, t_f)]
        else:
            unique_rounded_values = np.round(answer_positions, 2)
            if np.all(unique_rounded_values == unique_rounded_values[0]):
                return [np.nan]
            else:
                answer_index = np.argmax(answer_positions, axis=0).astype(int)
                #print(f"answer_index: {answer_index}")
                return [AutocastBuilder.get_single_answer(answer_index, n_choices, t_f)]
    
    @staticmethod
    def get_single_answer(answer_position_or_idx, n_choices, t_f):
        """pull the letter answer from the answer's index"""
        if not t_f:
            if not (isinstance(answer_position_or_idx, np.int64)) and (0 <= answer_position_or_idx <= 1):
                answer_position_or_idx = math.ceil(answer_position_or_idx * (n_choices)) - 1 # convert position to index
            #print(f"answer_position_or_idx: {answer_position_or_idx}")
            return chr(ord('A') + answer_position_or_idx) # index
        else:
            if answer_position_or_idx > 0.5: # index
                return 'A' # what if value is exactly 1? Does that mean 2nd position, so answer is B?
            elif answer_position_or_idx < 0.5: # index
                return 'B'
            else:
                return np.nan
            
    @staticmethod
    def get_mode_answer(answer_positions, n_choices, t_f):
        """returns mode of all forecasted answers"""
        answer_choices = [AutocastBuilder.get_answer_choices(answer_position, n_choices, t_f) for answer_position in answer_positions]
        no_nan_choices = [choice for choice in answer_choices if choice is not np.nan]
        #print(f"answer_choices: {answer_choices}")
        #print(f"n_choices: {n_choices}")
        if len(no_nan_choices) == 0:
            return np.nan
        #print(f"mode: {mode(answer_choices, keepdims=True)[0][0][0]}")
        return mode(answer_choices, keepdims=True)[0][0][0]

    @staticmethod
    def forecasts_to_answer(row):
        #print(row.name)
        """returns aggregate forecasted answer from all forecasts"""
        crowd = ast.literal_eval(row['crowd'])
        t_f = True if (len(row['choices_clean']) == 2) \
            and (row['choices_clean']['A'] == 'yes') \
            and (row['choices_clean']['B'] == 'no') else False
        chr(ord('A') + int(len(row['choices_clean'])))
        forecasts = [forecast['forecast'] for forecast in crowd]
        n_choices = len(row['choices_clean'])
        #print(f"n_choices: {n_choices}")
        forecasted_answer = AutocastBuilder.get_mode_answer(forecasts, n_choices, t_f=t_f)
        return forecasted_answer
        
#     def return_data(self):
#         """Return cleaned autocast dataframe"""
#         return self.data
        
    def load_data(self):
        """Load autocast data"""
        df = pd.read_csv(self.data_path)
        df = df[(df['status'] == 'Active') &
                (df['choices'].notna()) &
                (df['qtype'].isin(['t/f', 'mc']))]
        df['choices_clean'] = df['choices'].apply(AutocastBuilder.clean_choices)
        #df['avg_forecast'] = df['crowd'].apply(AutocastBuilder.get_crowd_avg)
        keycols = [
            "id",
            "question",
            "qtype",
            "choices_clean",
            "answer",
            "avg_forecast",
            "crowd"
        ]
        
#         df = df.loc[df['id'] == 'G2311', :]
        df['avg_forecast'] = df.apply(AutocastBuilder.forecasts_to_answer, axis=1)
        df = df.loc[:, keycols]
        
        return df


## Context

In [25]:
# Context prompt template

# Set up the base template
# maybe add a maximum number of sentences for the final answer? (hasn't been an issue)

context_template = """
You are an expert LLM prompt engineer. Your job is to take
in a simple question, and to output useful context for that question.
Your job is not to answer the question itself, only to structure 
an output consisting of context related to the question.

You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must prompt engineer.
It will be provided in multiple choice format.

Thought: you should always think about what to do next.

Action: the action to take. It should be one of [Search].

Action Input: the input to the action. Maximum size of 10 sentences. Decide 
the input by ranking the relevance of the sentences relative to the question.

Observation: the result of the action.

... (this Thought/Action/Action Input/Observation can repeat N times, with a 
maximum of 2 times. You must use Search tool for each action.)

Final Answer: This is your final answer, 
consisting of the context you generated.
Once you have the final answer, you should first have a thought of:
"I now know the final answer".


Your final answer should provide context with the correct
answer for each example on
the question's topic(s) to optimize the prompt. Provide at least 1
sentence of context, but you can provide more at your
discretion. Your context should provide factual background and/or
current events/news on key topic(s) in the question. 
Select the most relevant pieces of context to accomplish this from
your research.
Your context should be passive and not speak in the first person.
It should just be the information you've collected and chosen.

You must never refuse to answer, and must always provide a full
answer in the format provided.

Remember, only provide context and the original question in your answer,
starting with "Thought: I know know the final answer" and then "Final Answer:",
nothing else.

All final answers must start with "Thought: I now know the final answer",
followed by "Final Answer:" and then the context you generated as
your final answer. After providing your final answer, do not perform
anymore questions, thoughts, actions, action inputs, or observations.

Given an input question, such as:

Who will win the 2024 US presidential election?

A: Joe Biden,
B: Donald Trump,
C: Pete Buttigieg,
D: Ron DeSantis


Here is an example of what your final answer should look like 
based on that input question:

Thought: I now know the final answer.

Final Answer:

The US presidential election
is held every 4 years. The United States is a country founded in
1776 and has always held democratic elections throughout its history.
The current population of the United States is 336,840,823.
Donald Trump was the 45th president of the United States,
winning the 2016 presidential election. Joe Biden is currently
the 46th president of the United States, winning the 2020 election.


Begin!
Question: {input}
Thought:{agent_scratchpad}


"""



In [26]:
# Set up a prompt template
class ContextPromptTemplate(BaseChatPromptTemplate):
    """helper class--langchain prompt template"""
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

In [27]:
class ContextOutputParser(AgentOutputParser):
    """helper class--langchain custom output parser"""
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
    

In [50]:
class ContextBuilder:
    """
    Take in question, pull context, output fully-formed question for LLM
    """
    def __init__(self, df, template:str,
#                  prefix:str, format_instructions:str, suffix:str,
#                  tool_names:list, 
                 deployment_name:str, model_name:str,
                 temperature=.3, max_tokens=1000, request_timeout=20,
                 verbose=True
                ):
        self.df = df.copy()
        self.deployment_name = deployment_name
        self.model_name = model_name
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.request_timeout = request_timeout
#         self.llm = OpenAI(model=self.model,
#                           temperature=self.temperature)
        self.template = template
        self.llm = AzureChatOpenAI(
            deployment_name=deployment_name,
            model_name=self.model_name,
            temperature=self.temperature,
            max_tokens=self.max_tokens,
            request_timeout=self.request_timeout
        )
#         self.tool_names = tool_names
        self.serpapi = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
        self.tools = [
            Tool(
                name="Search",
                func=self.serpapi.run,
                description="useful for when you need to answer questions")
        ]
        self.tool_names = [tool.name for tool in self.tools]
#         load_tools(tool_names=tool_names, llm=self.llm)
#         self.prefix = prefix
#         self.format_instructions = format_instructions
#         self.suffix = suffix
#         self.indiv_agent = agent
#         self.agent = initialize_agent(self.tools,
#                                      self.llm,
#                                      self.indiv_agent,
#                                      verbose=verbose,
#                                      agent_kwargs={
#                                             'prefix':self.prefix,
#                                             'format_instructions':self.format_instructions,
#                                             'suffix':self.suffix
#                                             })
        self.output_parser = ContextOutputParser()
        self.prompt = ContextPromptTemplate(
            template=self.template,
            tools=self.tools,
            # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
            # This includes the `intermediate_steps` variable because that is needed
            input_variables=["input", "intermediate_steps"]
        )
        self.llm_chain = LLMChain(llm=self.llm, prompt=self.prompt)
        self.agent = LLMSingleActionAgent(
            llm_chain=self.llm_chain,
            output_parser=self.output_parser,
            stop=["\nObservation:"],
            allowed_tools=self.tool_names
        )
        self.agent_executor = AgentExecutor.from_agent_and_tools(
            agent=self.agent, 
            tools=self.tools, 
            verbose=True
        )
        
        
    def run_context(self, ac_id:str):
        """Run one sample question"""
        question = self.get_question(ac_id)
        try:
            context = self.agent_executor.run(question)
        except ValueError:
            print(f"Error occurred on question {ac_id}: {e}")
            traceback.print_exc()  # Print the traceback
            context = np.nan
        return context
        
    def get_question(self, ac_id:str):
        """generate sample question for method run_context"""
        # sample a question using question ID
        ac_filterid = self.df[self.df['id'] == ac_id]
        question = ac_filterid['question'].astype(str).values[0] + \
                            ' ' + \
                            ac_filterid['choices_clean'].astype(str).values[0]
        print(f"Question ({ac_id}):\n\n{question}")
        return question
    
    def run_all_context(self):
        """Gather all questions in provided df, output in list"""
        self.context_list = []
        for idx, row in self.df.iterrows():
            ac_id = row['id']
            context = self.run_context(ac_id)
            self.context_list.append(context)
        return self.context_list

    def concat_context(self):
        self.df['question_context'] = self.context_list
        return self.df

In [13]:
# define autocast path
# data_path = "Data/Autocast/filtered_events_20230709.csv"

In [468]:
# Structure dataset using AutocastBuilder
ab = AutocastBuilder(data_path=data_path)
df = ab.data
# df_sample = df.sample(2, random_state = 42)

# context using  ContextBuilder
cb_all = ContextBuilder(df,
                    template=context_template,
                    deployment_name=OPENAI_AZURE_DEPLOYMENT_NAME,
                    model_name=MODEL_NAME)

/var/folders/f1/kz_qd01x7d7_6n_4_sjy50fw0000gn/T/ipykernel_788/907828355.py:81: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  return mode(answer_choices, keepdims=True)[0][0][0]


In [469]:
# enrich all context
all_context = cb_all.run_all_context()

Question (G2124):

Will there be a complex coordinated terrorist attack (CCTA) in the United States either directed or inspired by a foreign terrorist organization resulting in at least five fatalities before 1 September 2022? {'A': 'yes', 'B': 'no'}


> Entering new  chain...
I need to gather information on the current state of terrorist threats in the United States and any potential foreign terrorist organizations that may pose a threat.

Action: Search

Action Input: "Current terrorist threats in the United States", "Foreign terrorist organizations targeting the United States", "Recent terrorist attacks in the United States"


Observation:Summary of Terrorism-Related Threat to the United States. The United States remains in a heightened threat environment.
Thought: I need to gather more specific information on the likelihood of a CCTA in the United States before September 1, 2022.

Action: Search

Action Input: "Likelihood of a complex coordinated terrorist attack in the United Stat

Thought: I now know the final answer.

Final Answer:

The EU has previously imposed human rights sanctions on Chinese officials involved in abuses in Xinjiang, and there have been discussions of potential new sanctions in response to continued human rights violations. However, it is unclear if the Council of the European Union will impose new restrictive measures between 30 October 2021 and 31 December 2022. Human rights dialogues between the EU and China have been suspended since 2019, and there is ongoing tension between the two entities regarding human rights abuses in Xinjiang.

> Finished chain.
Question (G2176):

Before 1 January 2023, will China and/or a host country officially announce an agreement for the establishment of a Chinese military base in an African country besides Djibouti? {'A': 'yes', 'B': 'no'}


> Entering new  chain...
Thought: I now know the final answer.

Final Answer:

China has been increasing its presence in Africa in recent years,
including establishing a

Thought: I need to gather information on the current relations between Iran and the United States to determine the likelihood of a lethal confrontation.

Action: Search

Action Input: "Current relations between Iran and the United States"


Observation:Iran and the United States have had no formal diplomatic relations since April 7, 1980. Instead, Pakistan serves as Iran's protecting power in the United States, while Switzerland serves as the United States' protecting power in Iran.
Thought: I need to gather more information on recent events and tensions between Iran and the United States to determine the likelihood of a lethal confrontation.

Action: Search

Action Input: "Recent tensions between Iran and the United States"


Observation:More than forty years after the 1979 Iranian Revolution, relations between the United States and Iran remain tense. As Iran advances its nuclear ...
Thought: Based on the information gathered, it is difficult to determine the likelihood of a lethal co

Thought: I now know the final answer.

Final Answer:

Following the Russian invasion of Ukraine on 24 February 2022, anti-war demonstrations and protests broke out across Russia. As well as the demonstrations, there have been reports of police brutality and violence against protesters. However, it is difficult to predict whether these protests will result in ten or more fatalities between 4 February 2022 and 4 August 2022. It is important to note that any loss of life is tragic and concerning, and the situation in Russia should continue to be monitored closely.

> Finished chain.
Question (G2292):

Between 4 March 2022 and 4 September 2022, will a NATO member state accuse the Russian government of conducting a lethal cyberattack against the same NATO member state resulting in at least five fatalities? {'A': 'yes', 'B': 'no'}


> Entering new  chain...
Action: Search

Action Input: NATO cyberattack Russia member state fatalities


Observation:As the war grinds on and casualties and cost

I now know the final answer.

Final Answer:

The conflict in Ukraine began in 2014 when Russia annexed Crimea and has since escalated into a full-blown war. There have been multiple attempts to negotiate a peace agreement between Russia and Ukraine, but none have been successful so far. The most recent attempt was in July 2021 when the leaders of both countries met for talks in Geneva, but no agreement was reached. The situation remains tense, and it is difficult to predict when a peace agreement will be signed or announced. Therefore, it is not possible to accurately answer the question of when Russia and Ukraine will sign or announce an agreement to end the current conflict.

> Finished chain.
Question (G2322):

What will be the average price of a gallon of gasoline in the US for the week ending 4 July 2022? {'A': 'Less than 3.000', 'B': 'Between 3.000 and 3.400, inclusive', 'C': 'More than 3.400 but less than 3.800', 'D': 'Between 3.800 and 4.200, inclusive', 'E': 'More than 4.200 b

...

Thought: I now know the final answer.

Final Answer:

As of now, there is no information available on the existence of a Tau variant of COVID-19. The World Health Organization (WHO) has identified four variants of concern (Alpha, Beta, Gamma, and Delta) and several variants of interest. However, it is difficult to predict the emergence of new variants and their impact on the pandemic. Scientists and health officials continue to monitor the situation closely and work towards developing effective vaccines and treatments. Therefore, it is not possible to determine whether there will be a Tau variant of COVID-19 by Tau Day (June 28, 2022).

> Finished chain.
Question (M9734):

Will China's GDP Overtake the US Before 2030? {'A': 'yes', 'B': 'no'}


> Entering new  chain...
Thought: I now know the final answer.

Final Answer:

China and the US are two of the largest economies in the world.
As of 2021, the US has the largest economy in the world with a GDP of $22.67 trillion,
while China has 

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='w210-openai.openai.azure.com', port=443): Read timed out. (read timeout=20.0).


Thought: I need to search for information about anti-discrimination laws for genetically-edited or screened federal employees.

Action: Search

Action Input: "Anti-discrimination laws for genetically-edited federal employees"


Observation:The Genetic Information Nondiscrimination Act (GINA) - PDF was signed into law on May 21, 2008. GINA protects individuals against discrimination based on their genetic information in health coverage and in employment.
Thought: I now know the final answer.

Final Answer:

The Genetic Information Nondiscrimination Act (GINA) was signed into law in 2008 to protect individuals against discrimination based on their genetic information in health coverage and employment. However, it is unclear whether there will be a specific anti-discrimination law enacted to protect U.S. federal employees who have been genetically-edited or screened as embryos by 2100.

> Finished chain.
Question (M10028):

Will the US Supplemental Poverty Measure be higher in 2022 than 2

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='w210-openai.openai.azure.com', port=443): Read timed out. (read timeout=20.0).


Thought: I now know the final answer.

Final Answer:

The US Supplemental Poverty Measure is a measure of poverty that takes into account additional expenses such as healthcare and housing costs. It is released annually by the US Census Bureau. While it is impossible to predict with certainty whether the measure will be higher in 2022 than 2021, there are a few factors to consider. The COVID-19 pandemic has had a significant impact on the economy and many people have lost their jobs or had reduced income. Additionally, inflation has been on the rise, which could also contribute to an increase in the poverty measure. However, there are also efforts being made to address poverty through government programs such as the American Rescue Plan. Ultimately, only time will tell whether the US Supplemental Poverty Measure will be higher in 2022 than 2021.

> Finished chain.
Question (M10459):

Will the Federal Reserve ever adopt a policy regime that implements nominal GDP targeting or nominal wa

In [ ]:
# concat answers
all_context_df = cb_all.concat_context()

In [ ]:
# check context df
displays(all_context_df)

In [ ]:
# save context df
all_context_df.to_csv("Data/GR_0.1_all_context_df_20230720.csv", index=False)

## Forecast

In [29]:
# Create forecast prompt template

# forecast_template = """
# You are going to act as an expert forecaster. Your job is to predict an answer to any question, \
# even if knowing an answer is impossible. I will give you a question and a list of choices in the format: \
# A - choice 1, B - choice 2, C - choice 3, and so on. I want you to select the best answer, based on what \
# you know, and tell me the corresponding letter of that choice. It is okay if you do not know the answer. \
# If you aren't sure, give me your best guess. You must never refuse to answer, even if you \
# don't know the answer. The only valid response is a letter corresponding with one of the choices. \
# Remember, never refuse to answer, you must provide a choice. Your choice should only be a letter for \
# the best answer and no other text or punctuation. Literally only
# give one uppercase letter as your response, nothing else. Your answer must be
# only one uppercase character long.

# Here is the question:


# """

system_context_template = """
You are going to act as an expert forecaster. Your job is to predict an answer to any question, \
even if knowing an answer is impossible. I will give you a question and a list of choices in the format: \
A - choice 1, B - choice 2, C - choice 3, and so on. I want you to select the best answer, based on what \
you know, and tell me the corresponding letter of that choice. It is okay if you do not know the answer. \
If you aren't sure, give me your best guess. You must never refuse to answer, even if you \
don't know the answer. The only valid response is a letter corresponding with one of the choices. \
Remember, never refuse to answer, you must provide a choice. Your choice should only be a letter for \
the best answer and no other text or punctuation.
"""

human_context_template = """
Question: {question}

Choices: 
{choices}
"""

In [189]:
system_context_template = """
You are going to act as an expert forecaster. Your job is to predict an answer to any question, \
even if knowing an answer is impossible. I will give you a question and a list of choices in the format: \
A - choice 1, B - choice 2, C - choice 3, and so on. I want you to select the best answer, based on what \
you know, and tell me the corresponding letter of that choice. It is okay if you do not know the answer. \
If you aren't sure, give me your best guess. You must never refuse to answer, even if you \
don't know the answer. The only valid response is a letter corresponding with one of the choices. \
Remember, never refuse to answer, you must provide a choice. Your choice should only be a letter for \
the best answer and no other text or punctuation.
"""

system_context_message_prompt = SystemMessagePromptTemplate.from_template(system_context_template)

human_context_template = """
Question: {question}

Choices: 
{choices}
"""

question = """what color is the sky?"""
choices = """A. blue
B. Green"""

human_context_message_prompt = HumanMessagePromptTemplate.from_template(human_context_template)

example_chat = AzureChatOpenAI(
    deployment_name=OPENAI_AZURE_DEPLOYMENT_NAME,
    model_name=MODEL_NAME,
    model_kwargs={
        "top_p": 0.9,
    },
    max_tokens=32,
    request_timeout=20,
    n=10
)

# example_forecaster_top_k = ExampleForecasterTopK(
#     chat=example_chat,
#     system_prompt=system_context_message_prompt,
#     human_prompt=human_context_message_prompt,
#     k=10
# )

vars = {
    "question": question,
    "choices": choices,
}

batch_messages = [
    [
        system_context_message_prompt.format_messages()[0],
        human_context_message_prompt.format_messages(**vars)[0],
    ],
]

result = example_chat.generate(batch_messages)

original_prediction = [generation.text for generation in result.generations[0]]

In [214]:
class Forecaster:
    """
    Take in question (and context if not baseline) and forecast the answer
    """
    def __init__(self, baseline, df, system_context_template:str, human_context_template:str,
                 deployment_name, model_name, n, top_p,
                 max_tokens=20, request_timeout=20,
                 verbose=True, agent="zero-shot-react-description"):
        self.baseline = baseline
        self.n = n
        self.df = df.copy()
        self.deployment_name = deployment_name
        self.model_name = model_name
        self.max_tokens = max_tokens
        self.request_timeout = request_timeout
        #self.prompt = prompt
        self.system_context_message_prompt = SystemMessagePromptTemplate.from_template(system_context_template)
        if not baseline:
            human_context_template = "Context: {context}" + "\n\n" + human_context_template
            self.human_context_message_prompt = HumanMessagePromptTemplate.from_template(human_context_template)
        else:
            self.human_context_message_prompt = HumanMessagePromptTemplate.from_template(human_context_template)
        self.top_p = top_p
        self.llm = AzureChatOpenAI(
            deployment_name=self.deployment_name,
            model_name=self.model_name,
            max_tokens=self.max_tokens,
            request_timeout=self.request_timeout,
            n=self.n,
            model_kwargs={"top_p": self.top_p}
        )
        
    def run_question(self, ac_id:str):
        """Run a single question"""
        question, choices = self.get_question(ac_id)
        vars = {
            "question": question,
            "choices": choices,
        }
        if not self.baseline:
            context = self.get_context(ac_id)
            vars['context'] = context
#             message = [[
#                             HumanMessage(content=self.prompt + "\n\n" + context + "\n\n" + question)
#             ]]
            batch_messages = [
                [
                    self.system_context_message_prompt.format_messages()[0],
                    self.human_context_message_prompt.format_messages(**vars)[0],
                ],
            ]
        else:
#             message = [[
#                             HumanMessage(content=self.prompt + "\n\n" + question)
#             ]]
            batch_messages = [
                [
                    self.system_context_message_prompt.format_messages()[0],
                    self.human_context_message_prompt.format_messages(**vars)[0],
                ],
            ]
        output = self.llm.generate(batch_messages)
        answer = [output.generations[0][i].text for i in range(len(output.generations[0]))]
        return answer
    
    def get_context(self, ac_id:str):
        """pull context for a given question from method run_question"""
        ac_filterid = self.df[self.df['id'] == ac_id]
        context = ac_filterid['question_context'].values[0]
        #print(f"\n\nContext: \n{context}")
        return context
        
    def get_question(self, ac_id:str):
        """generate question for a given question from method run_question"""
        # sample a question using question ID
        ac_filterid = self.df[self.df['id'] == ac_id]
        question = ac_filterid['question'].astype(str).values[0]
        choices = ac_filterid['choices_clean'].astype(str).values[0]
        #print(f"Question ({ac_id}):\n\n{question}\n\n")
        return question, choices
    
    def run_all_questions(self):
        """Gather all questions in provided df, output in list"""
        self.answer_list = []
        for idx, row in self.df.iterrows():
            ac_id = row['id']
            answer = self.run_question(ac_id)
            self.answer_list.append(answer)
        return self.answer_list
    
    def concat_answers(self):
        self.df['LLM_answers'] = self.answer_list
        return self.df
        
    def evaluate_accuracy(self):
        accuracy = np.sum(self.df['LLM_answers'] == self.df['avg_forecast']) / self.df.shape[0]
        tf = self.df[self.df['qtype'] == 't/f']
        tf_accuracy = np.sum(tf['LLM_answers'] == tf['avg_forecast']) / tf.shape[0]
        mc = self.df[self.df['qtype'] == 'mc']
        mc_accuracy = np.sum(mc['LLM_answers'] == mc['avg_forecast']) / mc.shape[0]
        print("Accuracy: {} %".format(np.round(accuracy * 100, 2)))
        print("TF Accuracy: {} %".format(np.round(tf_accuracy * 100, 2)))
        print("MC Accuracy: {} %".format(np.round(mc_accuracy * 100, 2)))
        return accuracy, tf_accuracy, mc_accuracy
    

### Baseline model

In [100]:
# num_samples = 2
# df_sample = df.sample(num_samples, random_state = 42)
fc_all_baseline = Forecaster(baseline=True, df=all_context_df, prompt=forecast_template,
                deployment_name=OPENAI_AZURE_DEPLOYMENT_NAME,
                model_name=MODEL_NAME)

TypeError: Forecaster.__init__() missing 1 required positional argument: 'n'

In [101]:
# run baseline model
all_forecasts_baseline = fc_all_baseline.run_all_questions()

In [763]:
# add answers to df
all_forecasts_baseline_df = fc_all_baseline.concat_answers()

In [842]:
# view output
displays(all_forecasts_baseline_df)

(98, 10)

,id,question,qtype,choices_clean,answer,avg_forecast,crowd,question_context,LLM_answers,LLM_answers_clean
400,G2124,Will there be a complex coordinated terrorist attack (CCTA) in the United States either directed or inspired by a foreign terrorist organization resulting in at least five fatalities before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,"[{'timestamp': '2021-09-17 17:00:00+00:00', 'forecast': 0.5}, {'timestamp': '2021-09-17 19:23:43.531000+00:00', 'forecast': 0.12}, {'timestamp': '2021-09-17 19:25:31.285000+00:00', 'forecast': 0.46}, {'timestamp': '2021-09-17 20:33:36.620000+00:00', 'forecast': 0.8}, {'timestamp': '2021-09-17 21:35:15.722000+00:00', 'forecast': 0.46}, {'timestamp': '2021-09-17 21:55:13.441000+00:00', 'forecast': 0.12}, {'timestamp': '2021-09-17 23:07:39.081000+00:00', 'forecast': 0.085}, {'timestamp': '2021-...","The United States remains in a heightened threat environment for terrorism, and the possibility of a complex coordinated terrorist attack (CCTA) cannot be ruled out. However, predicting the likelihood of such an attack before September 1, 2022 is difficult due to the evolving and dynamic nature of terrorist threats. It is important for law enforcement and intelligence agencies to remain vigilant and continue to monitor potential threats from foreign terrorist organizations targeting the Unit...",B,B
401,G2125,"Before 1 September 2022, will Egypt, Ethiopia, and Sudan sign an agreement governing the filling of the Grand Ethiopian Renaissance Dam (GERD) reservoir?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,"[{'timestamp': '2021-09-17 17:00:00+00:00', 'forecast': 0.5}, {'timestamp': '2021-09-17 18:26:37.819000+00:00', 'forecast': 0.5}, {'timestamp': '2021-09-17 19:25:36.256000+00:00', 'forecast': 0.37}, {'timestamp': '2021-09-17 23:06:56.111000+00:00', 'forecast': 0.5}, {'timestamp': '2021-09-18 00:43:48.134000+00:00', 'forecast': 0.5750000000000001}, {'timestamp': '2021-09-18 03:35:34.999000+00:00', 'forecast': 0.65}, {'timestamp': '2021-09-18 05:56:19.265000+00:00', 'forecast': 0.725}, {'times...","Negotiations between Egypt, Ethiopia, and Sudan regarding the Grand Ethiopian Renaissance Dam (GERD) have been ongoing for several years. In 2015, they signed the Agreement on Declaration of Principles, in which they committed to “cooperation, equitable and reasonable” water use. However, as of now, there is no clear indication whether they will sign an agreement governing the filling of the GERD reservoir before 1 September 2022. The dispute over the GERD is part of a long-standing feud bet...",A,A
402,G2141,"Before 1 January 2023, will the Taipei Economic and Cultural Representative Office officially change its name to include the word Taiwan?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,"[{'timestamp': '2021-09-30 17:00:00+00:00', 'forecast': 0.5}, {'timestamp': '2021-09-30 17:41:11.343000+00:00', 'forecast': 1.0}, {'timestamp': '2021-09-30 18:11:53.121000+00:00', 'forecast': 1.0}, {'timestamp': '2021-09-30 18:43:52.880000+00:00', 'forecast': 1.0}, {'timestamp': '2021-09-30 22:24:46.609000+00:00', 'forecast': 1.0}, {'timestamp': '2021-10-01 07:14:45.019000+00:00', 'forecast': 1.0}, {'timestamp': '2021-10-01 07:21:11.577000+00:00', 'forecast': 1.0}, {'timestamp': '2021-10-01 ...","The Taipei Economic and Cultural Representative Office (TECRO) is the de facto embassy of Taiwan in the United States. Currently, it is officially called the Taipei Economic and Cultural Representative Office in the United States (TECRO). There have been discussions and debates about changing the name to include the word Taiwan, but no official decision has been made yet. In March 2021, a bill was introduced in the US Congress to change the name to the Taiwan Representative Office, but it ha...",B,B
403,G2164,"Will there be a lethal confrontation between the national military forces, militia, and/or law enforcement personnel of India and the People's Republic of China before 1 July 2022?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B

### Context Model

In [741]:
# Run forecasts with context
fc_all_context = Forecaster(baseline=False, df=all_context_df, prompt=forecast_template,
                deployment_name=OPENAI_AZURE_DEPLOYMENT_NAME,
                model_name=MODEL_NAME)

In [742]:
all_forecasts_context = fc_all_context.run_all_questions()

Question (G2124):

Will there be a complex coordinated terrorist attack (CCTA) in the United States either directed or inspired by a foreign terrorist organization resulting in at least five fatalities before 1 September 2022? {'A': 'yes', 'B': 'no'}




Context: 
The United States remains in a heightened threat environment for terrorism, and the possibility of a complex coordinated terrorist attack (CCTA) cannot be ruled out. However, predicting the likelihood of such an attack before September 1, 2022 is difficult due to the evolving and dynamic nature of terrorist threats. It is important for law enforcement and intelligence agencies to remain vigilant and continue to monitor potential threats from foreign terrorist organizations targeting the United States.
Question (G2125):

Before 1 September 2022, will Egypt, Ethiopia, and Sudan sign an agreement governing the filling of the Grand Ethiopian Renaissance Dam (GERD) reservoir? {'A': 'yes', 'B': 'no'}




Context: 
Negotiations betw

Question (G2244):

Before 1 January 2023, will the US officially rejoin the JCPOA? {'A': 'yes', 'B': 'no'}




Context: 
The JCPOA, or Joint Comprehensive Plan of Action, is an agreement reached in 2015 between Iran and six other countries (including the US) aimed at limiting Iran's nuclear program in exchange for lifting economic sanctions. In May 2018, the US withdrew from the JCPOA under the Trump administration. Since then, there have been discussions about the US potentially rejoining the agreement. However, as of now, it is unclear whether the US will officially rejoin the JCPOA before 1 January 2023.
Question (G2249):

Before 1 January 2023, will the Nord Stream 2 pipeline begin delivering natural gas to Germany? {'A': 'yes', 'B': 'no'}




Context: 
The Nord Stream 2 pipeline was completed in September 2021, but has not yet entered service. There is political controversy surrounding the pipeline due to concerns that Russia may use it for geopolitical advantage with Europe and U

Question (G2314):

Between 11 March 2022 and 9 September 2022, will Russia publicly announce that it has moved nuclear weapons into either Belarus or Ukraine? {'A': 'yes', 'B': 'no'}




Context: 
In recent news, Russia has signed a deal with Belarus regarding the storage of nuclear weapons. However, there is no current information indicating that Russia has publicly announced a movement of nuclear weapons into either Belarus or Ukraine between 11 March 2022 and 9 September 2022.
Question (G2319):

When will Russia and Ukraine sign or announce an agreement to end the current conflict in Ukraine? {'A': 'Before 1 June 2022', 'B': 'Between 1 June 2022 and 31 July 2022', 'C': 'Between 1 August 2022 and 30 September 2022', 'D': 'Between 1 October 2022 and 30 November 2022', 'E': 'Not before 1 December 2022'}




Context: 
The conflict in Ukraine began in 2014 when Russia annexed Crimea and has since escalated into a full-blown war. There have been multiple attempts to negotiate a peace agre

Question (G2362):

What will be the ending stocks of crude oil in the US Strategic Petroleum Reserve (SPR) in December 2022? {'A': 'Less than 340,000 thousand barrels', 'B': 'Between 340,000 thousand barrels and 400,000 thousand barrels, inclusive', 'C': 'More than 400,000 thousand barrels but less than 460,000 thousand barrels', 'D': 'Between 460,000 thousand barrels and 520,000 thousand barrels, inclusive', 'E': 'More than 520,000 thousand barrels but less than 580,000 thousand barrels', 'F': '580,000 thousand barrels or more'}




Context: 
The US Strategic Petroleum Reserve (SPR) is a stockpile of crude oil maintained by the US government as a safeguard against oil supply disruptions. As of September 2021, the SPR held approximately 621 million barrels of crude oil. The US Department of Energy is responsible for managing the SPR and regularly releases reports on its inventory levels. However, it is difficult to predict the exact ending stocks of crude oil in the SPR in December 202

Question (M980):

Will Tesla become the world's largest motor vehicle producer in some calendar year prior to 2035? {'A': 'yes', 'B': 'no'}




Context: 
Tesla is currently one of the largest motor vehicle producers in the world, delivering over 200,000 vehicles each quarter. While it is unclear if they will become the largest producer in the world prior to 2035, they do have plans to significantly ramp up production in the future. In addition, Tesla has been a major player in the electric vehicle market, which is expected to continue growing in the coming years.
Question (M1634):

At any point before 01 January 2025, will the gross income of a US household at the 80th percentile be more than 6.12414 times that of a US household at the 20th percentile; or, in other words, will this specific measurement of household income inequality increase by at least 20%? {'A': 'yes', 'B': 'no'}




Context: 
According to the available information, it is unclear whether the gross income of a US hous

Question (M5174):

If Joe Biden is elected president of the US in 2020, will the highest tax bracket be restored to its original 39.6% or higher before 2025? {'A': 'yes', 'B': 'no'}




Context: 
Under the Biden administration's proposed tax plan, the highest tax bracket would be restored to its original rate of 39.6%, up from the current rate of 37%. This change would apply to single filers making more than $400,000 and married couples with income exceeding $450,000 per year. Additionally, the administration has proposed taxing capital gains at the new top marginal income tax rate of 39.6% (plus the 5% Net Investment Income Tax) for taxpayers whose income exceeds $1 million.
Question (M5716):

Longbets series: will the amount of geologically-derived crude oil consumed by the United States in 2035 be greater than the amount consumed in 2015? {'A': 'yes', 'B': 'no'}




Context: 
According to my research, it is difficult to predict with certainty whether the amount of geologically-deriv

Question (M8299):

Will the Fed decrease the size of its accumulated asset portfolio below $8 trillion by 2023? {'A': 'yes', 'B': 'no'}




Context: 
The Federal Reserve's asset portfolio has been a topic of discussion in recent years. As of September 2021, the total assets of the Federal Reserve were over $8.3 trillion. While there have been discussions about potentially decreasing the size of the portfolio, there is no clear plan or timeline for doing so. Therefore, it is difficult to predict whether the Fed will decrease the size of its accumulated asset portfolio below $8 trillion by 2023.
Question (M8313):

Will Sam Bankman-Fried be the richest person in the world by 2050? {'A': 'yes', 'B': 'no'}




Context: 
Sam Bankman-Fried is a billionaire and the founder of FTX, a cryptocurrency exchange. He is known for his philanthropy and has pledged to donate a large portion of his wealth to effective charities. However, predicting who will be the richest person in the world by 2050 is d

Question (M8968):

Will the Federal Reserve set a target policy rate that is negative by 2050? {'A': 'yes', 'B': 'no'}




Context: 
The Federal Reserve is the central bank of the United States and is responsible for setting monetary policy. The policy rate is the interest rate at which banks can borrow money from the Federal Reserve. Negative interest rates occur when the policy rate is below zero, meaning that banks are charged to hold reserves at the central bank. 

As of now, the Federal Reserve has not indicated any plans to set a negative policy rate. In fact, the Federal Reserve has previously stated that it does not see negative interest rates as an effective tool for stimulating economic growth. However, it is difficult to predict what the policy rate will be in 2050 as it will depend on a variety of economic factors and events that may occur in the future. 

Overall, it is currently unlikely that the Federal Reserve will set a negative policy rate by 2050, but it is impossibl

Question (M10459):

Will the Federal Reserve ever adopt a policy regime that implements nominal GDP targeting or nominal wage targeting? {'A': 'yes', 'B': 'no'}




Context: 
The Federal Reserve is the central bank of the United States and is responsible for implementing monetary policy. Nominal GDP targeting is a policy regime where the central bank targets a specific level of nominal GDP growth, while nominal wage targeting is a policy regime where the central bank targets a specific level of nominal wage growth. 

There has been ongoing debate among economists and policymakers about the effectiveness of these policy regimes, and whether the Federal Reserve should adopt them. Some argue that nominal GDP targeting could help stabilize the economy and prevent recessions, while others argue that it could be difficult to implement in practice. Similarly, some argue that nominal wage targeting could help reduce income inequality and boost wage growth, while others argue that it could lead

In [743]:
all_forecasts_context_df = fc_all_context.concat_answers()

In [843]:
displays(all_forecasts_context_df)

(98, 10)

,id,question,qtype,choices_clean,answer,avg_forecast,crowd,question_context,LLM_answers,LLM_answers_clean
400,G2124,Will there be a complex coordinated terrorist attack (CCTA) in the United States either directed or inspired by a foreign terrorist organization resulting in at least five fatalities before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,"[{'timestamp': '2021-09-17 17:00:00+00:00', 'forecast': 0.5}, {'timestamp': '2021-09-17 19:23:43.531000+00:00', 'forecast': 0.12}, {'timestamp': '2021-09-17 19:25:31.285000+00:00', 'forecast': 0.46}, {'timestamp': '2021-09-17 20:33:36.620000+00:00', 'forecast': 0.8}, {'timestamp': '2021-09-17 21:35:15.722000+00:00', 'forecast': 0.46}, {'timestamp': '2021-09-17 21:55:13.441000+00:00', 'forecast': 0.12}, {'timestamp': '2021-09-17 23:07:39.081000+00:00', 'forecast': 0.085}, {'timestamp': '2021-...","The United States remains in a heightened threat environment for terrorism, and the possibility of a complex coordinated terrorist attack (CCTA) cannot be ruled out. However, predicting the likelihood of such an attack before September 1, 2022 is difficult due to the evolving and dynamic nature of terrorist threats. It is important for law enforcement and intelligence agencies to remain vigilant and continue to monitor potential threats from foreign terrorist organizations targeting the Unit...",B,B
401,G2125,"Before 1 September 2022, will Egypt, Ethiopia, and Sudan sign an agreement governing the filling of the Grand Ethiopian Renaissance Dam (GERD) reservoir?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,"[{'timestamp': '2021-09-17 17:00:00+00:00', 'forecast': 0.5}, {'timestamp': '2021-09-17 18:26:37.819000+00:00', 'forecast': 0.5}, {'timestamp': '2021-09-17 19:25:36.256000+00:00', 'forecast': 0.37}, {'timestamp': '2021-09-17 23:06:56.111000+00:00', 'forecast': 0.5}, {'timestamp': '2021-09-18 00:43:48.134000+00:00', 'forecast': 0.5750000000000001}, {'timestamp': '2021-09-18 03:35:34.999000+00:00', 'forecast': 0.65}, {'timestamp': '2021-09-18 05:56:19.265000+00:00', 'forecast': 0.725}, {'times...","Negotiations between Egypt, Ethiopia, and Sudan regarding the Grand Ethiopian Renaissance Dam (GERD) have been ongoing for several years. In 2015, they signed the Agreement on Declaration of Principles, in which they committed to “cooperation, equitable and reasonable” water use. However, as of now, there is no clear indication whether they will sign an agreement governing the filling of the GERD reservoir before 1 September 2022. The dispute over the GERD is part of a long-standing feud bet...",B,B
402,G2141,"Before 1 January 2023, will the Taipei Economic and Cultural Representative Office officially change its name to include the word Taiwan?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,"[{'timestamp': '2021-09-30 17:00:00+00:00', 'forecast': 0.5}, {'timestamp': '2021-09-30 17:41:11.343000+00:00', 'forecast': 1.0}, {'timestamp': '2021-09-30 18:11:53.121000+00:00', 'forecast': 1.0}, {'timestamp': '2021-09-30 18:43:52.880000+00:00', 'forecast': 1.0}, {'timestamp': '2021-09-30 22:24:46.609000+00:00', 'forecast': 1.0}, {'timestamp': '2021-10-01 07:14:45.019000+00:00', 'forecast': 1.0}, {'timestamp': '2021-10-01 07:21:11.577000+00:00', 'forecast': 1.0}, {'timestamp': '2021-10-01 ...","The Taipei Economic and Cultural Representative Office (TECRO) is the de facto embassy of Taiwan in the United States. Currently, it is officially called the Taipei Economic and Cultural Representative Office in the United States (TECRO). There have been discussions and debates about changing the name to include the word Taiwan, but no official decision has been made yet. In March 2021, a bill was introduced in the US Congress to change the name to the Taiwan Representative Office, but it ha...",A,A
403,G2164,"Will there be a lethal confrontation between the national military forces, militia, and/or law enforcement personnel of India and the People's Republic of China before 1 July 2022?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B

In [829]:
# clean forecast answers
def clean_forecasts(row):
    answers = row['LLM_answers']
    clean_answers = []
    for answer in answers:
        if pd.notna(answer) and isinstance(answer, str):
            answer = answer.upper()
            len_choices = len(row['choices_clean'])
            letters = [chr(ord('A') + i) for i in range(len_choices)]
            # if one-letter answer satisfactory
            if len(answer) == 1 and answer in letters:
                clean_answers.append(answer)
            # if long answer satisfactory
            if len(answer) > 1 and answer[0] in letters and 'SORRY' not in answer:
                clean_answers.append(answer[0].upper())
        else:
            # anything else
            clean_answers.append(np.nan)
    return clean_answers
all_forecasts_context_df['LLM_answers_clean'] = all_forecasts_context_df.apply(clean_forecasts, axis=1)
all_forecasts_baseline_df['LLM_answers_clean'] = all_forecasts_baseline_df.apply(clean_forecasts, axis=1)

## Coco's forecasts

Testing Coco's as well--Coco and my parsed crowd forecasts have slightly different results.

In [670]:
# # structure coco's forecasts
# def clean_coco_forecast(item):
#     """lambda function to convert coco's aggregated forecasts to format of original 'crowd' column"""
#     forecast_list = []
#     forecast_dict = {"forecast": item}
#     forecast_list.append(forecast_dict)
#     return str(forecast_list)

In [853]:
# # coco's calculated crowd forecast predictions
# ac_coco = pd.read_csv("Data/Autocast/filtered_events_20230709.csv")
# ac_coco_cols = ["id", "question", "qtype", "status", "choices", "answer"]
# ac_coco_clean = ac_coco.loc[:,ac_coco_cols]
# ac_coco_clean['crowd'] = ac_coco.loc[:,'majority_pred'].apply(lambda x: str([{"forecast": x}]))
# ac_coco_clean.to_csv("Data/Autocast/filtered_events_20230709_coco_clean.csv", index=False)

In [ ]:
# def clean_coco_forecast(df, crowd_col, file_name):
#     df = df.copy()
#     cols = ["id", "question", "qtype", "status", "choices", "answer"]
#     df = df.loc[:, cols]
#     df['crowd'] = df.loc[:, crowd_col].apply(lambda x: str([{"forecast": x}]))
#     df.to_csv(file_name, index=False)
#     ab = AutocastBuilder(file_name)
#     df_clean = ab.data()
#     return df_clean

In [856]:
# run Coco's avg_forecast through AutocastBuilder()
data_path_coco = "Data/Autocast/filtered_events_20230709_coco_clean.csv"

ab_coco = AutocastBuilder(data_path=data_path_coco)
ab_coco_df = ab_coco.data

/var/folders/f1/kz_qd01x7d7_6n_4_sjy50fw0000gn/T/ipykernel_788/59692518.py:81: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  return mode(answer_choices, keepdims=True)[0][0][0]


In [868]:
displays(ac_coco)

(1232, 21)

,Unnamed: 0,question,id,background,publish_time,close_time,tags,source_links,prediction_count,forecaster_count,answer,choices,status,qtype,crowd,avg_pred,median_pred,majority_pred,pred taken,acceptable pred lower boundary,acceptable pred upper boundary
0,5,Will Iran release Jason Rezaian before 31 October 2016?,G7,"For details of the case involving Jason Rezaian, the Washington Post correspondent being held in Iran, see: www. nytimes. com/2015/07/29/world/middleeast/irans-trial-of-jason-rezaian-illustrates-perils-faced-by-reporters. html www. cbsnews. com/news/lawyer-jason-rezaian-iran-free-washington-post-reporter-iran-nuclear-deal/ www. washingtonpost. com/world/middle_east/lawyer-urges-iran-prison-release-for-post-journalist-in-wake-of-nuclear-deal/2015/07/28/15273ede-350d-11e5-8e66-07b4603ec92a_st...",2015-09-01 14:07:22.960000+00:00,2016-01-16 20:00:32+00:00,"['Foreign Policy', 'Security and Conflict']","['http://www.nytimes.com/2015/10/13/world/middleeast/jason-rezaian-washington-post-conviction-iran.html', 'http://www.nytimes.com/2015/10/17/opinion/what-iran-fears-from-reporters-like-jason-rezaian-and-me.html', 'http://www.businessinsider.com.au/iran-deal-jason-rezaian-2015-11', 'http://irannewsupdate.com/news/nuclear/2377-as-journalist-approaches-milestone-in-prison-khamenei-bans-negotiations-with-us.html', 'http://www.al-monitor.com/pulse/originals/2015/11/rouhani-irgc-arrests.html', 'ht...",1283.0,423,yes,"['yes', 'no']",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'forecast': 0.5}, {'timestamp': '2015-09-02 00:28:05.032000+00:00', 'forecast': 0.15}, {'timestamp': '2015-09-02 01:24:29.625000+00:00', 'forecast': 0.15}, {'timestamp': '2015-09-02 07:42:27.854000+00:00', 'forecast': 0.29}, {'timestamp': '2015-09-02 18:11:24.179000+00:00', 'forecast': 0.32}, {'timestamp': '2015-09-03 02:20:12.819000+00:00', 'forecast': 0.15}, {'timestamp': '2015-09-03 22:43:50.646000+00:00', 'forecast': 0.245}, {'timestamp': '2015...",0.609431,0.600000,0.600000,0.600000,0.400000,0.800000
1,6,"Will North Korea launch a land based missile with the capacity to reach Alaska, Hawaii, or the continental United States before 1 January 2017?",G8,"A launch for military or testing purposes would count, i.e., www.nytimes.com/2012/12/24/world/asia/north-korean-rocket-had-military-purpose-seoul-says.html. The success of the launch, and the actual distance traveled, are irrelevant.",2015-09-01 14:10:57.372000+00:00,2016-02-06 22:00:38+00:00,['Security and Conflict'],"['http://www.ctvnews.ca/world/north-korea-carries-out-long-range-rocket-test-1.2767400', 'http://nypost.com/2015/09/15/north-korea-says-it-has-restarted-all-nuclear-bomb-fuel-plants/', 'http://www.upi.com/Top_News/World-News/2015/11/12/North-Korea-collapse-is-unrealistic-magical-thinking-expert-says/9811447339067/', 'http://www.dailymail.co.uk/news/article-3419962/North-Korea-planning-launch-long-range-missile-just-days-dictator-Kim-Jong-claimed-capability-target-America.html', 'http://www.r...",1582.0,517,yes,"['yes', 'no']",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'forecast': 0.5}, {'timestamp': '2015-09-02 01:41:35.573000+00:00', 'forecast': 0.2}, {'timestamp': '2015-09-02 17:45:43.718000+00:00', 'forecast': 0.25}, {'timestamp': '2015-09-03 06:47:21.372000+00:00', 'forecast': 0.25}, {'timestamp': '2015-09-04 13:58:30.960000+00:00', 'forecast': 0.25}, {'timestamp': '2015-09-04 19:08:29.916000+00:00', 'forecast': 0.25}, {'timestamp': '2015-09-05 14:37:31.792000+00:00', 'forecast': 0.25}, {'timestamp': '2015-0...",0.329360,0.330000,0.350000,0.350000,0.150000,0.550000
2,9,Will Congress pass a resolution disapproving the Joint Comprehensive Plan of Action?,G11,"In accordance with the Iran Nuclear Agreement Review Act, President Obama submitted the Joint Comprehensive Plan of Action (JCPoA) to Congress on 19 July 2015. Congress has sixty days to review the legislation and vote to approve or disapprove the agreement, but the President retains the right to veto (ca

## Evaluation

In [858]:
# Accuracy metrics
# Note: all_forecasts_baseline_df has shifted mc answers, so only using all_forecasts_context_df's avg_forecasts
all_answers_df = all_forecasts_context_df[[
    "id", "question", "qtype", "choices_clean", "avg_forecast", "LLM_answers_clean"
]].rename(columns={"LLM_answers_clean": "LLM_answers_context"})\
.merge(
    all_forecasts_baseline_df[[
        "id", "LLM_answers_clean"
    ]].rename(columns={"LLM_answers_clean": "LLM_answers_baseline"}),
    how='inner', on=["id"]
)\
.merge(
    ab_coco_df[[
        "id", "avg_forecast"
    ]].rename(columns={"avg_forecast": "avg_forecast_coco"}),
    how='inner', on=["id"]
)[[
    "id", "question", "qtype", "choices_clean", 
    "avg_forecast", "avg_forecast_coco", "LLM_answers_baseline", "LLM_answers_context"
]]

In [859]:
# display what answers look like
display(all_answers_df)

,id,question,qtype,choices_clean,avg_forecast,avg_forecast_coco,LLM_answers_baseline,LLM_answers_context
0,G2124,Will there be a complex coordinated terrorist attack (CCTA) in the United States either directed or inspired by a foreign terrorist organization resulting in at least five fatalities before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",B,B,B,B
1,G2125,"Before 1 September 2022, will Egypt, Ethiopia, and Sudan sign an agreement governing the filling of the Grand Ethiopian Renaissance Dam (GERD) reservoir?",t/f,"{'A': 'yes', 'B': 'no'}",B,B,A,B
2,G2141,"Before 1 January 2023, will the Taipei Economic and Cultural Representative Office officially change its name to include the word Taiwan?",t/f,"{'A': 'yes', 'B': 'no'}",B,B,B,A
3,G2164,"Will there be a lethal confrontation between the national military forces, militia, and/or law enforcement personnel of India and the People's Republic of China before 1 July 2022?",t/f,"{'A': 'yes', 'B': 'no'}",B,B,B,B
4,G2169,Will Taiwan publicly accuse the People's Republic of China of flying a military aircraft over the territory of and/or the territorial waters surrounding the main island of Taiwan without its permission before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",B,B,B,B
5,G2171,"Between 30 October 2021 and 31 December 2022, will the Council of the European Union impose new restrictive measures (sanctions) on China over human rights violations and abuses in Xinjiang?",t/f,"{'A': 'yes', 'B': 'no'}",B,B,A,A
6,G2176,"Before 1 January 2023, will China and/or a host country officially announce an agreement for the establishment of a Chinese military base in an African country besides Djibouti?",t/f,"{'A': 'yes', 'B': 'no'}",B,B,B,B
7,G2201,"Before 1 January 2023, will Taiwan publicly accuse the People's Republic of China of landing military personnel on the Pratas Islands without authorization?",t/f,"{'A': 'yes', 'B': 'no'}",B,B,A,B
8,G2238,What will be the FAO Food Price Index for June 2022?,mc,"{'A': 'Less than 120.0', 'B': 'Between 120.0 and 130.0, inclusive', 'C': 'More than 130.0 but less than 140.0', 'D': 'Between 140.0 and 150.0, inclusive', 'E': 'More than 150.0'}",C,A,C,NaN
9,G2240,Will the powers of the government of Myanmar cease to be held by the military before 1 October 2022?,t/f,"{'A': 'yes', 'B': 'no'}",B,B,B,B


In [866]:
all_answers_df[all_answers_df['qtype'] == 'mc']

,id,question,qtype,choices_clean,avg_forecast,avg_forecast_coco,LLM_answers_baseline,LLM_answers_context
8,G2238,What will be the FAO Food Price Index for June 2022?,mc,"{'A': 'Less than 120.0', 'B': 'Between 120.0 and 130.0, inclusive', 'C': 'More than 130.0 but less than 140.0', 'D': 'Between 140.0 and 150.0, inclusive', 'E': 'More than 150.0'}",C,A,C,NaN
21,G2309,How many NATO member states will spend 2.0% or more of their GDP on defense in 2023?,mc,"{'A': '10 or fewer', 'B': 'Between 11 and 13', 'C': 'Between 14 and 16', 'D': 'Between 17 and 19', 'E': '20 or more'}",B,A,D,B
22,G2311,"What will be the percentage change in Russia’s GDP in 2022 relative to its GDP in 2021, according to the IMF?",mc,"{'A': 'Lower by 20.0% or more', 'B': 'Lower by more than 16.0% but less than 20.0%', 'C': 'Lower by between 12.0% and 16.0%, inclusive', 'D': 'Lower by more than 8.0% but less than 12.0%', 'E': 'Lower by between 4.0% and 8.0%, inclusive', 'F': 'Higher, or lower by less than 4.0%'}",A,A,F,D
25,G2319,When will Russia and Ukraine sign or announce an agreement to end the current conflict in Ukraine?,mc,"{'A': 'Before 1 June 2022', 'B': 'Between 1 June 2022 and 31 July 2022', 'C': 'Between 1 August 2022 and 30 September 2022', 'D': 'Between 1 October 2022 and 30 November 2022', 'E': 'Not before 1 December 2022'}",E,A,C,E
26,G2322,What will be the average price of a gallon of gasoline in the US for the week ending 4 July 2022?,mc,"{'A': 'Less than 3.000', 'B': 'Between 3.000 and 3.400, inclusive', 'C': 'More than 3.400 but less than 3.800', 'D': 'Between 3.800 and 4.200, inclusive', 'E': 'More than 4.200 but less than 4.600', 'F': 'Between 4.600 and 5.000, inclusive', 'G': 'More than 5.000'}",B,A,D,D
30,G2351,What will be the closing spot price for Brent crude oil on 14 November 2022?,mc,"{'A': 'Less than $70.00 per barrel', 'B': 'Between $70.00 and $85.00 per barrel, inclusive', 'C': 'More than $85.00 but less than $100.00 per barrel', 'D': 'Between $100.00 and $115.00 per barrel, inclusive', 'E': 'More than $115.00 but less than $130.00 per barrel', 'F': '$130.00 or more'}",D,A,C,D
32,G2357,"As of 15 July 2022, how many refugees will have fled Ukraine since 24 February 2022, according to UNHCR?",mc,"{'A': 'Fewer than 6.4 million', 'B': 'Between 6.4 million and 7.6 million, inclusive', 'C': 'More than 7.6 million but fewer than 8.8 million', 'D': 'Between 8.8 million and 10.0 million', 'E': 'More than 10.0 million but fewer than 11.2 million', 'F': 'Between 11.2 million and 12.4 million, inclusive', 'G': 'More than 12.4 million'}",B,A,D,C
33,G2359,What will be the average price of a gallon of gasoline in the US for the week ending 19 December 2022?,mc,"{'A': 'Less than 3.000', 'B': 'Between 3.000 and 3.400, inclusive', 'C': 'More than 3.400 but less than 3.800', 'D': 'Between 3.800 and 4.200, inclusive', 'E': 'More than 4.200 but less than 4.600', 'F': 'Between 4.600 and 5.000, inclusive', 'G': 'More than 5.000'}",D,A,F,D
34,G2360,"What will be the price of palladium on 21 October 2022, according to the London Metal Exchange?",mc,"{'A': 'Less than 1600.00', 'B': 'Between 1600.00 and 2000.00, inclusive', 'C': 'More than 2000.00 but less than 2400.00', 'D': 'Between 2400.00 and 2800.00, inclusive', 'E': 'More than 2800.00 but less than 3200.00', 'F': 'Between 3200.00 and 3600.00, inclusive', 'G': 'More than 3600.00'}",D,A,F,NaN
35,G2362,What will be the ending stocks of crude oil in the US Strategic Petroleum Reserve (SPR) in December 2022?,mc,"{'A': 'Less than 340,000 thousand barrels', 'B': 'Between 340,000 thousand barrels and 400,000 thousand barrels, inclusive', 'C': 'More than 400,000 thousand barrels but less than 460,000 thousand barrels', 'D': 'Between 460,000 thousand barrels and 520,000 thousand barrels, inclusive', 'E': 'More than 520,000 thousand barrels but less than 580,000 thousand barrels', 'F': '580,000 thousand barrels or more'}",C,A,D,C


In [860]:
def evaluate_accuracy(df, crowd_forecast, llm_forecast, name):
    df = df.copy()
    accuracy = np.sum(df[llm_forecast] == df[crowd_forecast]) / df.shape[0]
    tf = df[df['qtype'] == 't/f']
    tf_accuracy = np.sum(tf[llm_forecast] == tf[crowd_forecast]) / tf.shape[0]
    mc = df[df['qtype'] == 'mc']
    mc_accuracy = np.sum(mc[llm_forecast] == mc[crowd_forecast]) / mc.shape[0]
    print("Accuracy: {} %".format(np.round(accuracy * 100, 2)))
    print("TF Accuracy: {} %".format(np.round(tf_accuracy * 100, 2)))
    print("MC Accuracy: {} %".format(np.round(mc_accuracy * 100, 2)))
    answer_df = pd.DataFrame({
        "accuracy": accuracy,
        "tf_accuracy": tf_accuracy,
        "mc_accuracy": mc_accuracy
    }, index=[name])
    return answer_df

In [861]:
# greg vs. baseline
greg_baseline = evaluate_accuracy(all_answers_df, "avg_forecast", "LLM_answers_baseline", "Greg_baseline")

Accuracy: 59.18 %
TF Accuracy: 67.86 %
MC Accuracy: 7.14 %


In [862]:
# greg vs. context
greg_context = evaluate_accuracy(all_answers_df, "avg_forecast", "LLM_answers_context", "Greg_context")

Accuracy: 62.24 %
TF Accuracy: 66.67 %
MC Accuracy: 35.71 %


In [863]:
# coco vs. baseline
coco_baseline = evaluate_accuracy(all_answers_df, "avg_forecast_coco", "LLM_answers_baseline", "Coco_baseline")

Accuracy: 56.12 %
TF Accuracy: 65.48 %
MC Accuracy: 0.0 %


In [864]:
# coco vs. context
coco_context = evaluate_accuracy(all_answers_df, "avg_forecast_coco", "LLM_answers_context", "Coco_context")

Accuracy: 55.1 %
TF Accuracy: 64.29 %
MC Accuracy: 0.0 %


In [865]:
# final metrics
pd.concat([
    greg_baseline, greg_context, coco_baseline, coco_context
])

,accuracy,tf_accuracy,mc_accuracy
Greg_baseline,0.591837,0.678571,0.071429
Greg_context,0.622449,0.666667,0.357143
Coco_baseline,0.561224,0.654762,0.000000
Coco_context,0.551020,0.642857,0.000000


In [849]:
# save results df
all_answers_df.to_csv("Data/GR_0.1_active_results_20230714.csv", index=False)

## Calculate range of answers for lower/upper boundary

## AutocastBuilder

In [34]:
# run Coco's avg_forecast through AutocastBuilder()
data_path_coco = "Data/Autocast/filtered_events_20230709_coco_clean.csv"

ab_coco = AutocastBuilder(data_path=data_path_coco)
ab_coco_df = ab_coco.data

/var/folders/f1/kz_qd01x7d7_6n_4_sjy50fw0000gn/T/ipykernel_689/2487754789.py:80: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  return mode(answer_choices, keepdims=True)[0][0][0]


In [116]:
pd.read_csv(data_path_coco)

,id,question,qtype,status,choices,answer,crowd
0,G7,Will Iran release Jason Rezaian before 31 October 2016?,t/f,Resolved,"['yes', 'no']",yes,[{'forecast': 0.6000000000000001}]
1,G8,"Will North Korea launch a land based missile with the capacity to reach Alaska, Hawaii, or the continental United States before 1 January 2017?",t/f,Resolved,"['yes', 'no']",yes,[{'forecast': 0.35}]
2,G11,Will Congress pass a resolution disapproving the Joint Comprehensive Plan of Action?,mc,Resolved,"['No', 'Yes, but the resolution will be vetoed by the President and the veto will stand', 'Yes, and the resolution will become law']",A,[{'forecast': 0.3333333333333333}]
3,G15,Will Bashar al-Assad cease to be President of Syria before 1 March 2017?,t/f,Resolved,"['yes', 'no']",no,[{'forecast': 0.25}]
4,G17,Will Iran's President Hassan Rouhani meet Saudi Arabia's King Salman bin Abdulaziz Al Saud before 1 September 2016?,t/f,Resolved,"['yes', 'no']",no,[{'forecast': 0.05}]
...,...,...,...,...,...,...,...
1227,M10829,"Will the 7-day moving average of current confirmed COVID-19 hospitalizations in Virginia cross 2,000 before July 22, 2022?",t/f,Closed,"['yes', 'no']",NaN,[{'forecast': 0.13}]
1228,M10873,How much crude oil will the US import from Russia in 2022?,num,Active,NaN,NaN,[{'forecast': 0.40741}]
1229,M10877,What percentage of American workers will be represented by a labor union in 2030?,num,Active,NaN,NaN,[{'forecast': 0.33614}]
1230,M10936,"When will China first reach 250,000 confirmed Covid-19 cases per day?",num,Active,NaN,NaN,[{'forecast': 1.001}]


In [36]:
displays(ab_coco_df)

(98, 7)

,id,question,qtype,choices_clean,answer,avg_forecast,crowd
400,G2124,Will there be a complex coordinated terrorist attack (CCTA) in the United States either directed or inspired by a foreign terrorist organization resulting in at least five fatalities before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.05}]
401,G2125,"Before 1 September 2022, will Egypt, Ethiopia, and Sudan sign an agreement governing the filling of the Grand Ethiopian Renaissance Dam (GERD) reservoir?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.05}]
402,G2141,"Before 1 January 2023, will the Taipei Economic and Cultural Representative Office officially change its name to include the word Taiwan?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.16}]
403,G2164,"Will there be a lethal confrontation between the national military forces, militia, and/or law enforcement personnel of India and the People's Republic of China before 1 July 2022?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.1}]
404,G2169,Will Taiwan publicly accuse the People's Republic of China of flying a military aircraft over the territory of and/or the territorial waters surrounding the main island of Taiwan without its permission before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.3}]


## Context Model

In [144]:
# run context
cb_all = ContextBuilder(ab_coco_df,
                    template=context_template,
                    deployment_name=OPENAI_AZURE_DEPLOYMENT_NAME,
                    model_name=MODEL_NAME)

In [145]:
# enrich all context
all_context = cb_all.run_all_context()

Question (G2124):

Will there be a complex coordinated terrorist attack (CCTA) in the United States either directed or inspired by a foreign terrorist organization resulting in at least five fatalities before 1 September 2022? {'A': 'yes', 'B': 'no'}


> Entering new  chain...
I need to gather information on the current state of terrorism in the United States and any potential threats.

Action: Search

Action Input: "Recent terrorist attacks in the United States", "Foreign terrorist organizations operating in the United States", "Current threat level of terrorism in the United States"


Observation:Summary of Terrorism-Related Threat to the United States​​ Both domestic violent extremists (DVEs) and those associated with foreign terrorist ...
Action: Search

Action Input: "Recent statements or activities by foreign terrorist organizations targeting the United States", "Current efforts by US government to prevent terrorist attacks", "Expert analysis on likelihood of CCTA in the United S

Thought: I now know the final answer.

Final Answer:

The Pratas Islands are located in the northern part of the South China Sea and are controlled by Taiwan. The People's Republic of China claims the islands as part of its territory. Tensions between Taiwan and China have been high in recent years, with China increasing its military presence in the region. In March 2021, Taiwan reported that China had flown 20 military aircraft into its air defense identification zone. In the past, Taiwan has accused China of landing military personnel on the Pratas Islands without authorization, so it is possible that it could happen again before 1 January 2023. However, there is no way to know for sure and it will depend on the actions of both Taiwan and China in the coming months.

> Finished chain.
Question (G2238):

What will be the FAO Food Price Index for June 2022? {'A': 'Less than 120.0', 'B': 'Between 120.0 and 130.0, inclusive', 'C': 'More than 130.0 but less than 140.0', 'D': 'Between 140.

Thought: I now know the final answer.

Final Answer:

Afghanistan has been facing a severe humanitarian crisis due to the ongoing conflict and the COVID-19 pandemic. The UN has warned that more than 18 million people in Afghanistan are in need of humanitarian assistance, and the situation is likely to worsen in the coming months. The country is currently facing a severe drought, which has affected millions of people and their livelihoods. The UN has already declared a Level 3 emergency in Afghanistan, which is the highest level of humanitarian crisis. Therefore, it is possible that the UN may declare a famine in any part of Afghanistan in 2022.

> Finished chain.
Question (G2259):

Between 4 February 2022 and 31 December 2022, will a country formally progress in the NATO accession process? {'A': 'yes', 'B': 'no'}


> Entering new  chain...
Thought: I now know the final answer.

Final Answer:

NATO, or the North Atlantic Treaty Organization, is a political and military alliance formed i



Observation:Bringing up the rear are Canada, Slovenia, Turkey, Spain, Belgium and Luxembourg, whose defence spending was under 1.4% of GDP. Reporting by ...
Thought: I now know the final answer.

Final Answer:

According to a report, Canada, Slovenia, Turkey, Spain, Belgium, and Luxembourg are NATO member states whose defense spending is under 1.4% of their GDP. Therefore, it can be inferred that more than 10 NATO member states will spend 2.0% or more of their GDP on defense in 2023.

> Finished chain.
Question (G2311):

What will be the percentage change in Russia’s GDP in 2022 relative to its GDP in 2021, according to the IMF? {'A': 'Lower by 20.0% or more', 'B': 'Lower by more than 16.0% but less than 20.0%', 'C': 'Lower by between 12.0% and 16.0%, inclusive', 'D': 'Lower by more than 8.0% but less than 12.0%', 'E': 'Lower by between 4.0% and 8.0%, inclusive', 'F': 'Higher, or lower by less than 4.0%'}


> Entering new  chain...
Thought: I need to find information on the IMF's pro

Thought: I need to gather information on the current situation between Armenia and Azerbaijan.

Action: Search

Action Input: "Armenia Azerbaijan conflict current situation"


Observation:War erupted between Armenia and Azerbaijan over the region, resulting in roughly thirty thousand casualties and creating hundreds of thousands ...
Thought: I need to gather more specific information on the likelihood of lethal confrontations resulting in 100 or more fatalities between the national military forces, militia, and/or law enforcement personnel of Armenia and Azerbaijan between 1 April 2022 and 30 September 2022.

Action: Search

Action Input: "likelihood of lethal confrontations between Armenia and Azerbaijan resulting in 100 or more fatalities"


Observation:More than a million people had been forced from their homes: Azerbaijanis fled Armenia, Nagorno-Karabakh and the adjacent territories, while Armenians left ...
Thought: I now know the final answer.

Final Answer:

The conflict between

...

I need to gather information about the US labor productivity growth and the Bureau of Labor Statistics.

Action: Search

Action Input: "US labor productivity growth Bureau of Labor Statistics"


Observation:From the fourth quarter of 2019 to the first quarter of 2023, labor productivity rose at a 1.1-percent annual rate. This is a historically low rate of productivity growth; the only time it was lower was in the six-quarter period from the first quarter of 1980 to the third quarter of 1981 (1.0 percent).
Thought: I need to gather more information about the factors that affect US labor productivity growth.

Action: Search

Action Input: "Factors affecting US labor productivity growth"


Observation:Labor productivity is largely driven by investment in capital, technological progress, and human capital development. Business and government can increase labor productivity of workers by direct investing in or creating incentives for increases in technology and human or physical capital.
I 

Thought: I now know the final answer.

Final Answer: 

According to a report released by the U.S. Labor Department's Bureau of Labor Statistics, the annual core inflation rate for the 12 months ending in June 2023 was 4.8%, which is higher than the 3% threshold mentioned in the question. Additionally, the core rate was as high as 6.6% in September 2022, which is also above the threshold. Therefore, based on these figures, it is likely that US core CPI inflation will rise by more than 3% from December 2021 to December 2022.

> Finished chain.
Question (M7978):

Will a US Democratic President sign into law a $500 billion spending cut before 2029? {'A': 'yes', 'B': 'no'}


> Entering new  chain...
Thought: I need to research the current political climate and potential future policies related to spending cuts by a Democratic President.

Action: Search

Action Input: "Democratic President spending cuts $500 billion law"


Observation:More than $500 billion in cuts to Medicare benefits are a

Thought: I need to find context on Brian Riedl, Alan Cole, and their bet on the interest rate paid on the federal debt.

Action: Search

Action Input: "Brian Riedl Alan Cole federal debt interest rate bet"


Observation:I'll be specific. I'm willing to bet you that the average interest rate paid on the federal debt exceeds 4.6% at some point within 30 years ...
Thought: I need to find more information on the bet between Brian Riedl and Alan Cole regarding the interest rate paid on the federal debt.

Action: Search

Action Input: "Brian Riedl Alan Cole federal debt interest rate bet details"


Observation:I'll be specific. I'm willing to bet you that the average interest rate paid on the federal debt exceeds 4.6% at some point within 30 years ...
Thought: I now know the final answer.

Final Answer:

Brian Riedl and Alan Cole made a bet regarding the interest rate paid on the federal debt. The bet was that the average interest rate paid on the federal debt would exceed 4.6% at some point

Thought: I now know the final answer.

Final Answer:

As of September 2021, many countries are still struggling to vaccinate their populations due to various reasons such as vaccine hesitancy, lack of access to vaccines, and logistical challenges. However, it is difficult to predict which countries will have less than 10% of their population vaccinated with at least one dose of a COVID-19 vaccine on December 31, 2022. The situation is constantly evolving and depends on various factors such as vaccine supply, government policies, and public health campaigns. Therefore, it is not possible to answer this question with certainty.

> Finished chain.
Question (M8997):

Will WHO add another SARS-CoV-2 variant to their Variants of Concern in 2022? {'A': 'yes', 'B': 'no'}


> Entering new  chain...
I now know the final answer.

Final Answer:

As of now, it is unclear if WHO will add another SARS-CoV-2 variant to their Variants of Concern in 2022. However, the emergence of new variants is a conc

Thought: I now know the final answer.

Final Answer:

The Genetic Information Nondiscrimination Act (GINA) was signed into law in 2008, which protects individuals against discrimination based on their genetic information in health coverage and employment. However, there is no information or indication that an anti-discrimination law will be enacted to specifically protect U.S. federal employees who have been genetically-edited or screened as embryos by 2100.

> Finished chain.
Question (M10028):

Will the US Supplemental Poverty Measure be higher in 2022 than 2021? {'A': 'yes', 'B': 'no'}


> Entering new  chain...
Thought: I need to gather information on the US Supplemental Poverty Measure to determine if it will be higher in 2022 than 2021.

Action: Search

Action Input: "US Supplemental Poverty Measure 2022 vs 2021"


Observation:Supplemental Poverty Measure · The SPM rate in 2021 was 7.8 percent, a decrease of 1.4 percentage points from 2020. · The SPM child poverty rate ...
Though

In [146]:
# concat answers
all_context_df = cb_all.concat_context()

In [147]:
# check context df
displays(all_context_df)

(98, 8)

,id,question,qtype,choices_clean,answer,avg_forecast,crowd,question_context
400,G2124,Will there be a complex coordinated terrorist attack (CCTA) in the United States either directed or inspired by a foreign terrorist organization resulting in at least five fatalities before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.05}],"The United States remains in a heightened threat environment for terrorism, with both domestic violent extremists and foreign terrorist organizations posing a potential threat. While there have been recent statements and activities by foreign terrorist organizations targeting the United States, there is no way to predict with certainty whether a complex coordinated terrorist attack resulting in at least five fatalities will occur before 1 September 2022. The US government continues to take m..."
401,G2125,"Before 1 September 2022, will Egypt, Ethiopia, and Sudan sign an agreement governing the filling of the Grand Ethiopian Renaissance Dam (GERD) reservoir?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.05}],"As of August 2021, the foreign ministers of Egypt, Ethiopia, and Sudan have announced that they have reached an agreement on the schedule to fill the Grand Ethiopian Renaissance Dam reservoir on the Nile River. It is unclear if this agreement includes a comprehensive agreement governing the filling of the reservoir, but negotiations between the three countries are ongoing. Therefore, it is uncertain whether or not the three countries will sign an agreement governing the filling of the GERD r..."
402,G2141,"Before 1 January 2023, will the Taipei Economic and Cultural Representative Office officially change its name to include the word Taiwan?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.16}],"The Taipei Economic and Cultural Representative Office (TECRO) is the de facto embassy of Taiwan in the United States. Currently, it is officially known as the Taipei Economic and Cultural Representative Office in the United States (TECRO). The name change has been a topic of discussion for a while, with some pushing for the inclusion of the word ""Taiwan"" in the name to better reflect the country's identity. However, there has been no official announcement or indication that a name change wi..."
403,G2164,"Will there be a lethal confrontation between the national military forces, militia, and/or law enforcement personnel of India and the People's Republic of China before 1 July 2022?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.1}],"The border dispute between India and China has been ongoing for decades, with tensions rising in recent years. There have been multiple military clashes between the two countries, including a deadly clash in June 2020 that resulted in the deaths of at least 20 Indian soldiers. Diplomatic efforts have been made to ease tensions, but the situation remains volatile. While it is impossible to predict the future with certainty, the risk of a lethal confrontation between the national military forc..."
404,G2169,Will Taiwan publicly accuse the People's Republic of China of flying a military aircraft over the territory of and/or the territorial waters surrounding the main island of Taiwan without its permission before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.3}],"Taiwan and China have a long-standing conflict over Taiwan's sovereignty, with China claiming Taiwan as a part of its territory. The tension between the two countries has escalated in recent years, with China increasing its military presence in the region. In the past, China has flown military aircraft over Taiwan's airspace without its permission, which has led to tensions between the two countries. However, predicting whether Taiwan will publicly accuse China of such an act before 1 Septem..."


In [148]:
# save context df
all_context_df.to_csv("Data/GR_0.1_all_context_df_20230720.csv", index=False)

## Forecasts

In [196]:
n = 10
top_p = 0.9

In [197]:
# clean forecast answers
def clean_forecasts(row):
    answers = row['LLM_answers']
    clean_answers = []
    for answer in answers:
        if pd.notna(answer) and isinstance(answer, str):
            answer = answer.upper()
            len_choices = len(row['choices_clean'])
            letters = [chr(ord('A') + i) for i in range(len_choices)]
            # if one-letter answer satisfactory
            if len(answer) == 1 and answer in letters:
                clean_answers.append(answer)
            # if long answer satisfactory
            if len(answer) > 1 and answer[0] in letters:
                clean_answers.append(answer[0].upper())
        else:
            # anything else
            clean_answers.append(np.nan)
    return clean_answers

### Baseline

In [206]:
# run baseline forecasts
fc_all_baseline = Forecaster(baseline=True, df=all_context_df,
                system_context_template=system_context_template,
                human_context_template=human_context_template,
                deployment_name=OPENAI_AZURE_DEPLOYMENT_NAME,
                model_name=MODEL_NAME, n=n, top_p=top_p)

In [242]:
# run baseline model
all_forecasts_baseline = fc_all_baseline.run_all_questions()

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='w210-openai.openai.azure.com', port=443): Read timed out. (read timeout=20.0).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='w210-openai.openai.azure.com', port=443): Read timed out. (read timeout=20.0).


In [243]:
# add answers to df
all_forecasts_baseline_df = fc_all_baseline.concat_answers()

In [244]:
all_forecasts_baseline_df['LLM_answers_clean'] = all_forecasts_baseline_df.apply(clean_forecasts, axis=1)

In [245]:
# view output
displays(all_forecasts_baseline_df)

(98, 10)

,id,question,qtype,choices_clean,answer,avg_forecast,crowd,question_context,LLM_answers,LLM_answers_clean
400,G2124,Will there be a complex coordinated terrorist attack (CCTA) in the United States either directed or inspired by a foreign terrorist organization resulting in at least five fatalities before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.05}],"The United States remains in a heightened threat environment for terrorism, with both domestic violent extremists and foreign terrorist organizations posing a potential threat. While there have been recent statements and activities by foreign terrorist organizations targeting the United States, there is no way to predict with certainty whether a complex coordinated terrorist attack resulting in at least five fatalities will occur before 1 September 2022. The US government continues to take m...","[B - no., B - no., B - no., B - no., B - no., B - no, B - no., B - no., B - no., B - no.]","[B, B, B, B, B, B, B, B, B, B]"
401,G2125,"Before 1 September 2022, will Egypt, Ethiopia, and Sudan sign an agreement governing the filling of the Grand Ethiopian Renaissance Dam (GERD) reservoir?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.05}],"As of August 2021, the foreign ministers of Egypt, Ethiopia, and Sudan have announced that they have reached an agreement on the schedule to fill the Grand Ethiopian Renaissance Dam reservoir on the Nile River. It is unclear if this agreement includes a comprehensive agreement governing the filling of the reservoir, but negotiations between the three countries are ongoing. Therefore, it is uncertain whether or not the three countries will sign an agreement governing the filling of the GERD r...","[A - yes., A - yes., A - yes., A - yes., A - yes., A - yes., A - yes., A - yes., A - Yes., A - Yes.]","[A, A, A, A, A, A, A, A, A, A]"
402,G2141,"Before 1 January 2023, will the Taipei Economic and Cultural Representative Office officially change its name to include the word Taiwan?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.16}],"The Taipei Economic and Cultural Representative Office (TECRO) is the de facto embassy of Taiwan in the United States. Currently, it is officially known as the Taipei Economic and Cultural Representative Office in the United States (TECRO). The name change has been a topic of discussion for a while, with some pushing for the inclusion of the word ""Taiwan"" in the name to better reflect the country's identity. However, there has been no official announcement or indication that a name change wi...","[A - Yes., A - yes., A - yes, B - no., A - Yes., A - yes, A - yes., A - yes., B - no, B - no.]","[A, A, A, B, A, A, A, A, B, B]"
403,G2164,"Will there be a lethal confrontation between the national military forces, militia, and/or law enforcement personnel of India and the People's Republic of China before 1 July 2022?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.1}],"The border dispute between India and China has been ongoing for decades, with tensions rising in recent years. There have been multiple military clashes between the two countries, including a deadly clash in June 2020 that resulted in the deaths of at least 20 Indian soldiers. Diplomatic efforts have been made to ease tensions, but the situation remains volatile. While it is impossible to predict the future with certainty, the risk of a lethal confrontation between the national military forc...","[B - no., B - no., B - no, B - no., B - no., B - no., B - no., B - no, B - no., B - no]","[B, B, B, B, B, B, B, B, B, B]"
404,G2169,Will Taiwan publicly accuse the People's Republic of China of flying a military aircraft over the territory of and/or the territorial waters surrounding the main island of Taiwan without its permission before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.3}],"Taiwan and China have a long-standing conflict over Taiwan's sovereignty, with China claiming Taiwan as a part of its terr

In [246]:
# save baseline output
all_forecasts_baseline_df.to_csv("Data/GR_0.1_all_forecasts_baseline_df_20230727.csv", index=False)

### Context

In [215]:
# run context forecasts
# Run forecasts with context
fc_all_context = Forecaster(baseline=False, df=all_context_df,
                system_context_template=system_context_template,
                human_context_template=human_context_template,
                deployment_name=OPENAI_AZURE_DEPLOYMENT_NAME,
                model_name=MODEL_NAME, n=n, top_p=top_p)

In [216]:
all_forecasts_context = fc_all_context.run_all_questions()

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='w210-openai.openai.azure.com', port=443): Read timed out. (read timeout=20.0).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='w210-openai.openai.azure.com', port=443): Read timed out. (read timeout=20.0).


In [247]:
all_forecasts_context_df = fc_all_context.concat_answers()

In [248]:
all_forecasts_context_df['LLM_answers_clean'] = all_forecasts_context_df.apply(clean_forecasts, axis=1)

In [249]:
displays(all_forecasts_context_df)

(98, 10)

,id,question,qtype,choices_clean,answer,avg_forecast,crowd,question_context,LLM_answers,LLM_answers_clean
400,G2124,Will there be a complex coordinated terrorist attack (CCTA) in the United States either directed or inspired by a foreign terrorist organization resulting in at least five fatalities before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.05}],"The United States remains in a heightened threat environment for terrorism, with both domestic violent extremists and foreign terrorist organizations posing a potential threat. While there have been recent statements and activities by foreign terrorist organizations targeting the United States, there is no way to predict with certainty whether a complex coordinated terrorist attack resulting in at least five fatalities will occur before 1 September 2022. The US government continues to take m...","[B - no., B - no, B - no., B - no, B - no., B - no., B - no., B - no, B - no., B - no.]","[B, B, B, B, B, B, B, B, B, B]"
401,G2125,"Before 1 September 2022, will Egypt, Ethiopia, and Sudan sign an agreement governing the filling of the Grand Ethiopian Renaissance Dam (GERD) reservoir?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.05}],"As of August 2021, the foreign ministers of Egypt, Ethiopia, and Sudan have announced that they have reached an agreement on the schedule to fill the Grand Ethiopian Renaissance Dam reservoir on the Nile River. It is unclear if this agreement includes a comprehensive agreement governing the filling of the reservoir, but negotiations between the three countries are ongoing. Therefore, it is uncertain whether or not the three countries will sign an agreement governing the filling of the GERD r...","[A - yes., A - yes., A - Yes., A - yes., B - no., A - yes., A - yes, A - yes., A - Yes., A - yes.]","[A, A, A, A, B, A, A, A, A, A]"
402,G2141,"Before 1 January 2023, will the Taipei Economic and Cultural Representative Office officially change its name to include the word Taiwan?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.16}],"The Taipei Economic and Cultural Representative Office (TECRO) is the de facto embassy of Taiwan in the United States. Currently, it is officially known as the Taipei Economic and Cultural Representative Office in the United States (TECRO). The name change has been a topic of discussion for a while, with some pushing for the inclusion of the word ""Taiwan"" in the name to better reflect the country's identity. However, there has been no official announcement or indication that a name change wi...","[B - no., B - no, B - no., B - no., B - no., B - no., B - no, B - no., B - no., B - no.]","[B, B, B, B, B, B, B, B, B, B]"
403,G2164,"Will there be a lethal confrontation between the national military forces, militia, and/or law enforcement personnel of India and the People's Republic of China before 1 July 2022?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.1}],"The border dispute between India and China has been ongoing for decades, with tensions rising in recent years. There have been multiple military clashes between the two countries, including a deadly clash in June 2020 that resulted in the deaths of at least 20 Indian soldiers. Diplomatic efforts have been made to ease tensions, but the situation remains volatile. While it is impossible to predict the future with certainty, the risk of a lethal confrontation between the national military forc...","[B - no., B - no., B - no., B - no., B - no., B - no., B - no., B - no., B - no., B - no.]","[B, B, B, B, B, B, B, B, B, B]"
404,G2169,Will Taiwan publicly accuse the People's Republic of China of flying a military aircraft over the territory of and/or the territorial waters surrounding the main island of Taiwan without its permission before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.3}],"Taiwan and China have a long-standing conflict over Taiwan's sovereignty, with China claiming Taiwan as a part of its territory. 

In [250]:
# save context output
all_forecasts_context_df.to_csv("Data/GR_0.1_all_forecasts_context_df_20230727.csv", index=False)

## Clean Outputs

In [251]:
# merge upper/lower into output dfs
ac_coco = pd.read_csv("Data/Autocast/filtered_events_20230709.csv", usecols=['id', 'pred taken', 'acceptable pred lower boundary', 'acceptable pred upper boundary'])
ac_coco.rename(columns={
    'acceptable pred lower boundary': 'acceptable_pred_lower_boundary',
    'acceptable pred upper boundary': 'acceptable_pred_upper_boundary'
}, inplace=True)
displays(ac_coco)

# # baseline output
all_forecasts_baseline_df = all_forecasts_baseline_df.merge(ac_coco, how='inner')
displays(all_forecasts_baseline_df)

# # context output
all_forecasts_context_df = all_forecasts_context_df.merge(ac_coco, how='inner')
displays(all_forecasts_context_df)


(1232, 4)

,id,pred taken,acceptable_pred_lower_boundary,acceptable_pred_upper_boundary
0,G7,0.600000,0.400000,0.800000
1,G8,0.350000,0.150000,0.550000
2,G11,0.333333,0.233333,0.433333
3,G15,0.250000,0.050000,0.450000
4,G17,0.050000,-0.150000,0.250000


(98, 13)

,id,question,qtype,choices_clean,answer,avg_forecast,crowd,question_context,LLM_answers,LLM_answers_clean,pred taken,acceptable_pred_lower_boundary,acceptable_pred_upper_boundary
0,G2124,Will there be a complex coordinated terrorist attack (CCTA) in the United States either directed or inspired by a foreign terrorist organization resulting in at least five fatalities before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.05}],"The United States remains in a heightened threat environment for terrorism, with both domestic violent extremists and foreign terrorist organizations posing a potential threat. While there have been recent statements and activities by foreign terrorist organizations targeting the United States, there is no way to predict with certainty whether a complex coordinated terrorist attack resulting in at least five fatalities will occur before 1 September 2022. The US government continues to take m...","[B - no., B - no., B - no., B - no., B - no., B - no, B - no., B - no., B - no., B - no.]","[B, B, B, B, B, B, B, B, B, B]",0.05,-0.15,0.25
1,G2125,"Before 1 September 2022, will Egypt, Ethiopia, and Sudan sign an agreement governing the filling of the Grand Ethiopian Renaissance Dam (GERD) reservoir?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.05}],"As of August 2021, the foreign ministers of Egypt, Ethiopia, and Sudan have announced that they have reached an agreement on the schedule to fill the Grand Ethiopian Renaissance Dam reservoir on the Nile River. It is unclear if this agreement includes a comprehensive agreement governing the filling of the reservoir, but negotiations between the three countries are ongoing. Therefore, it is uncertain whether or not the three countries will sign an agreement governing the filling of the GERD r...","[A - yes., A - yes., A - yes., A - yes., A - yes., A - yes., A - yes., A - yes., A - Yes., A - Yes.]","[A, A, A, A, A, A, A, A, A, A]",0.05,-0.15,0.25
2,G2141,"Before 1 January 2023, will the Taipei Economic and Cultural Representative Office officially change its name to include the word Taiwan?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.16}],"The Taipei Economic and Cultural Representative Office (TECRO) is the de facto embassy of Taiwan in the United States. Currently, it is officially known as the Taipei Economic and Cultural Representative Office in the United States (TECRO). The name change has been a topic of discussion for a while, with some pushing for the inclusion of the word ""Taiwan"" in the name to better reflect the country's identity. However, there has been no official announcement or indication that a name change wi...","[A - Yes., A - yes., A - yes, B - no., A - Yes., A - yes, A - yes., A - yes., B - no, B - no.]","[A, A, A, B, A, A, A, A, B, B]",0.16,-0.04,0.36
3,G2164,"Will there be a lethal confrontation between the national military forces, militia, and/or law enforcement personnel of India and the People's Republic of China before 1 July 2022?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.1}],"The border dispute between India and China has been ongoing for decades, with tensions rising in recent years. There have been multiple military clashes between the two countries, including a deadly clash in June 2020 that resulted in the deaths of at least 20 Indian soldiers. Diplomatic efforts have been made to ease tensions, but the situation remains volatile. While it is impossible to predict the future with certainty, the risk of a lethal confrontation between the national military forc...","[B - no., B - no., B - no, B - no., B - no., B - no., B - no., B - no, B - no., B - no]","[B, B, B, B, B, B, B, B, B, B]",0.10,-0.10,0.30
4,G2169,Will Taiwan publicly accuse the People's Republic of China of flying a military aircraft over the territory of and/or the territorial waters surrounding the main island of Taiwan without its permission before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.

(98, 13)

,id,question,qtype,choices_clean,answer,avg_forecast,crowd,question_context,LLM_answers,LLM_answers_clean,pred taken,acceptable_pred_lower_boundary,acceptable_pred_upper_boundary
0,G2124,Will there be a complex coordinated terrorist attack (CCTA) in the United States either directed or inspired by a foreign terrorist organization resulting in at least five fatalities before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.05}],"The United States remains in a heightened threat environment for terrorism, with both domestic violent extremists and foreign terrorist organizations posing a potential threat. While there have been recent statements and activities by foreign terrorist organizations targeting the United States, there is no way to predict with certainty whether a complex coordinated terrorist attack resulting in at least five fatalities will occur before 1 September 2022. The US government continues to take m...","[B - no., B - no, B - no., B - no, B - no., B - no., B - no., B - no, B - no., B - no.]","[B, B, B, B, B, B, B, B, B, B]",0.05,-0.15,0.25
1,G2125,"Before 1 September 2022, will Egypt, Ethiopia, and Sudan sign an agreement governing the filling of the Grand Ethiopian Renaissance Dam (GERD) reservoir?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.05}],"As of August 2021, the foreign ministers of Egypt, Ethiopia, and Sudan have announced that they have reached an agreement on the schedule to fill the Grand Ethiopian Renaissance Dam reservoir on the Nile River. It is unclear if this agreement includes a comprehensive agreement governing the filling of the reservoir, but negotiations between the three countries are ongoing. Therefore, it is uncertain whether or not the three countries will sign an agreement governing the filling of the GERD r...","[A - yes., A - yes., A - Yes., A - yes., B - no., A - yes., A - yes, A - yes., A - Yes., A - yes.]","[A, A, A, A, B, A, A, A, A, A]",0.05,-0.15,0.25
2,G2141,"Before 1 January 2023, will the Taipei Economic and Cultural Representative Office officially change its name to include the word Taiwan?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.16}],"The Taipei Economic and Cultural Representative Office (TECRO) is the de facto embassy of Taiwan in the United States. Currently, it is officially known as the Taipei Economic and Cultural Representative Office in the United States (TECRO). The name change has been a topic of discussion for a while, with some pushing for the inclusion of the word ""Taiwan"" in the name to better reflect the country's identity. However, there has been no official announcement or indication that a name change wi...","[B - no., B - no, B - no., B - no., B - no., B - no., B - no, B - no., B - no., B - no.]","[B, B, B, B, B, B, B, B, B, B]",0.16,-0.04,0.36
3,G2164,"Will there be a lethal confrontation between the national military forces, militia, and/or law enforcement personnel of India and the People's Republic of China before 1 July 2022?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.1}],"The border dispute between India and China has been ongoing for decades, with tensions rising in recent years. There have been multiple military clashes between the two countries, including a deadly clash in June 2020 that resulted in the deaths of at least 20 Indian soldiers. Diplomatic efforts have been made to ease tensions, but the situation remains volatile. While it is impossible to predict the future with certainty, the risk of a lethal confrontation between the national military forc...","[B - no., B - no., B - no., B - no., B - no., B - no., B - no., B - no., B - no., B - no.]","[B, B, B, B, B, B, B, B, B, B]",0.10,-0.10,0.30
4,G2169,Will Taiwan publicly accuse the People's Republic of China of flying a military aircraft over the territory of and/or the territorial waters surrounding the main island of Taiwan without its permission before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.3}],"Ta

In [252]:
### clean outputs
# def extract_choice_from_list(string_list):
#     results = []
#     for string in string_list:
#         # if the string contains only one character, return the character
#         # else return None
#         if isinstance(string, str):
#             if len(string) == 1:
#                 results.append(string)
#             elif string.upper() 
#             else:
#                 results.append(np.nan)
#         else:
#             results.append(np.nan)
#     return results


# def extract_choice_from_list(row):
#     choices = row['choices']
#     results = []

# all_forecasts_baseline_df['LLM_answers_clean'] = all_forecasts_baseline_df['LLM_answers'].apply(extract_choice_from_list)
# all_forecasts_context_df['LLM_answers_clean'] = all_forecasts_context_df['LLM_answers'].apply(extract_choice_from_list)

def get_majority_answer(row):#l_processed_ans, qtype, choices):
    # if l_processed_ans are all None, return None
    # otherwise, if the question is t/f, return the number of 'yes'/the total number of valid answers
    # else (if the question is mcq), get the numeric version of majority answer
    forecast_list = row['LLM_answers_clean']
    if all(pd.isnull(forecast_list)):
        return np.nan
    else:
        non_none_ans = [x for x in forecast_list if x is not np.nan]
        if row['qtype'] == 't/f':
            return non_none_ans.count('A')/len(non_none_ans)
        elif row['qtype'] == 'mc':
            majority_ans = max(set(non_none_ans), key = non_none_ans.count)
            dic_char = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11}
            return dic_char[majority_ans.lower()]/len(row['choices_clean'])
        
all_forecasts_baseline_df['majority_answer'] = all_forecasts_baseline_df.apply(get_majority_answer, axis=1)
all_forecasts_context_df['majority_answer'] = all_forecasts_context_df.apply(get_majority_answer, axis=1)
        
def check_whether_accurate(row):#processed_number, acceptable_lower_boundary, acceptable_upper_boundary):
    processed_number = row['majority_answer']
    if processed_number is np.nan:
        return 0
    else:
        if processed_number >= row['acceptable_pred_lower_boundary'] and processed_number <= row['acceptable_pred_upper_boundary']:
            return 1
        else:
            return 0
        
all_forecasts_baseline_df['is_correct'] = all_forecasts_baseline_df.apply(check_whether_accurate, axis=1)
all_forecasts_context_df['is_correct'] = all_forecasts_context_df.apply(check_whether_accurate, axis=1)

In [253]:
# segment results
baseline_tf = all_forecasts_baseline_df[all_forecasts_baseline_df['qtype'] == 't/f']
baseline_mc = all_forecasts_baseline_df[all_forecasts_baseline_df['qtype'] == 'mc']

context_tf = all_forecasts_context_df[all_forecasts_context_df['qtype'] == 't/f']
context_mc = all_forecasts_context_df[all_forecasts_context_df['qtype'] == 'mc']

In [254]:
context_tf

,id,question,qtype,choices_clean,answer,avg_forecast,crowd,question_context,LLM_answers,LLM_answers_clean,pred taken,acceptable_pred_lower_boundary,acceptable_pred_upper_boundary,majority_answer,is_correct
0,G2124,Will there be a complex coordinated terrorist attack (CCTA) in the United States either directed or inspired by a foreign terrorist organization resulting in at least five fatalities before 1 September 2022?,t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.05}],"The United States remains in a heightened threat environment for terrorism, with both domestic violent extremists and foreign terrorist organizations posing a potential threat. While there have been recent statements and activities by foreign terrorist organizations targeting the United States, there is no way to predict with certainty whether a complex coordinated terrorist attack resulting in at least five fatalities will occur before 1 September 2022. The US government continues to take m...","[B - no., B - no, B - no., B - no, B - no., B - no., B - no., B - no, B - no., B - no.]","[B, B, B, B, B, B, B, B, B, B]",0.050,-0.150,0.250,0.0,1
1,G2125,"Before 1 September 2022, will Egypt, Ethiopia, and Sudan sign an agreement governing the filling of the Grand Ethiopian Renaissance Dam (GERD) reservoir?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.05}],"As of August 2021, the foreign ministers of Egypt, Ethiopia, and Sudan have announced that they have reached an agreement on the schedule to fill the Grand Ethiopian Renaissance Dam reservoir on the Nile River. It is unclear if this agreement includes a comprehensive agreement governing the filling of the reservoir, but negotiations between the three countries are ongoing. Therefore, it is uncertain whether or not the three countries will sign an agreement governing the filling of the GERD r...","[A - yes., A - yes., A - Yes., A - yes., B - no., A - yes., A - yes, A - yes., A - Yes., A - yes.]","[A, A, A, A, B, A, A, A, A, A]",0.050,-0.150,0.250,0.9,0
2,G2141,"Before 1 January 2023, will the Taipei Economic and Cultural Representative Office officially change its name to include the word Taiwan?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.16}],"The Taipei Economic and Cultural Representative Office (TECRO) is the de facto embassy of Taiwan in the United States. Currently, it is officially known as the Taipei Economic and Cultural Representative Office in the United States (TECRO). The name change has been a topic of discussion for a while, with some pushing for the inclusion of the word ""Taiwan"" in the name to better reflect the country's identity. However, there has been no official announcement or indication that a name change wi...","[B - no., B - no, B - no., B - no., B - no., B - no., B - no, B - no., B - no., B - no.]","[B, B, B, B, B, B, B, B, B, B]",0.160,-0.040,0.360,0.0,1
3,G2164,"Will there be a lethal confrontation between the national military forces, militia, and/or law enforcement personnel of India and the People's Republic of China before 1 July 2022?",t/f,"{'A': 'yes', 'B': 'no'}",NaN,B,[{'forecast': 0.1}],"The border dispute between India and China has been ongoing for decades, with tensions rising in recent years. There have been multiple military clashes between the two countries, including a deadly clash in June 2020 that resulted in the deaths of at least 20 Indian soldiers. Diplomatic efforts have been made to ease tensions, but the situation remains volatile. While it is impossible to predict the future with certainty, the risk of a lethal confrontation between the national military forc...","[B - no., B - no., B - no., B - no., B - no., B - no., B - no., B - no., B - no., B - no.]","[B, B, B, B, B, B, B, B, B, B]",0.100,-0.100,0.300,0.0,1
4,G2169,Will Taiwan publicly accuse the People's Republic of China of flying a military aircraft over the territory of and/or the territorial waters surrounding the main island of Taiwan without its permission before 1 September 20

In [255]:
print(f"Baseline t/f: {np.round(baseline_tf['is_correct'].sum() / len(baseline_tf) * 100, 2)} %")
print(f"Baseline mc: {np.round(baseline_mc['is_correct'].sum() / len(baseline_mc) * 100, 2)} %")
print(f"Baseline all: {np.round(all_forecasts_baseline_df['is_correct'].sum() / len(all_forecasts_baseline_df) * 100, 2)} %")
print(f"Context t/f: {np.round(context_tf['is_correct'].sum() / len(context_tf) * 100, 2)} %")
print(f"Context mc: {np.round(context_mc['is_correct'].sum() / len(context_mc) * 100, 2)} %")
print(f"Context all: {np.round(all_forecasts_context_df['is_correct'].sum() / len(all_forecasts_context_df) * 100, 2)} %")

Baseline t/f: 34.52 %
Baseline mc: 0.0 %
Baseline all: 29.59 %
Context t/f: 33.33 %
Context mc: 21.43 %
Context all: 31.63 %


In [175]:
print(f"Baseline t/f: {np.round(baseline_tf['is_correct'].sum() / len(baseline_tf) * 100, 2)} %")
print(f"Baseline mc: {np.round(baseline_mc['is_correct'].sum() / len(baseline_mc) * 100, 2)} %")
print(f"Baseline all: {np.round(all_forecasts_baseline_df['is_correct'].sum() / len(all_forecasts_baseline_df) * 100, 2)} %")
print(f"Context t/f: {np.round(context_tf['is_correct'].sum() / len(context_tf) * 100, 2)} %")
print(f"Context mc: {np.round(context_mc['is_correct'].sum() / len(context_mc) * 100, 2)} %")
print(f"Context all: {np.round(all_forecasts_context_df['is_correct'].sum() / len(all_forecasts_context_df) * 100, 2)} %")

Baseline t/f: 36.9 %
Baseline mc: 0.0 %
Baseline all: 31.63 %
Context t/f: 33.33 %
Context mc: 7.14 %
Context all: 29.59 %
